In [388]:
import numpy as np
import pandas as pd
import re
import datetime
import openpyxl
import calendar
import os
import glob
from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.enum.shapes import MSO_SHAPE
import matplotlib.pyplot as plt
import io
from functools import reduce
import math
import psutil

In [389]:
''' 
Список МО для выгрузки из отчета ЦБ Главное - Дополнительные отчеты - показатели записи на прием к врачу за период в 1 неделю которые у меня называются record
ГАУЗ ТО "Городская поликлиника № 1"; ГАУЗ ТО "Городская поликлиника № 12"; ГАУЗ ТО "Городская поликлиника № 13 "; ГАУЗ ТО "Городская поликлиника № 4"; ГАУЗ ТО "Городская поликлиника №5"; ГАУЗ ТО "Областная больница № 19" (г.Тюмень); ГАУЗ ТО «Городская Поликлиника № 6»; ГАУЗ ТО «Городская поликлиника № 8»; ГАУЗ ТО «Городская поликлиника №17»; ГАУЗ ТО «Городская поликлиника №3»; ГБУЗ ТО "Областная больница № 11" (р.п. Голышманово); ГБУЗ ТО "Областная больница № 12" (г.Заводоуковск); ГБУЗ ТО "Областная больница № 14 им. В.Н.Шанаурина" (с. Казанское); ГБУЗ ТО "Областная больница № 15" (с.Нижняя Тавда); ГБУЗ ТО "Областная больница № 23" (г.Ялуторовск); ГБУЗ ТО "Областная больница № 24" (с.Ярково); ГБУЗ ТО "Областная больница № 3" (г.Тобольск); ГБУЗ ТО "Областная больница № 4" (г.Ишим); ГБУЗ ТО "Областная больница № 9" (с.Вагай); ГБУЗ ТО "Областная больница №13" (с. Исетское); ГБУЗ ТО "Областная стоматологическая поликлиника"; ГБУЗ ТО "ОКБ №2"; ГБУЗ ТО "ОНД"; ГБУЗ ТО "Роддом № 3"; ГБУЗ ТО "Роддом №2"; ГБУЗ ТО «Областной клинический фтизиопульмонологический центр»; ГБУЗ ТО Областная больница №20 (с.Уват); ГБУЗ ТО ОКПБ
'''

' \nСписок МО для выгрузки из отчета ЦБ Главное - Дополнительные отчеты - показатели записи на прием к врачу за период в 1 неделю которые у меня называются record\nГАУЗ ТО "Городская поликлиника № 1"; ГАУЗ ТО "Городская поликлиника № 12"; ГАУЗ ТО "Городская поликлиника № 13 "; ГАУЗ ТО "Городская поликлиника № 4"; ГАУЗ ТО "Городская поликлиника №5"; ГАУЗ ТО "Областная больница № 19" (г.Тюмень); ГАУЗ ТО «Городская Поликлиника № 6»; ГАУЗ ТО «Городская поликлиника № 8»; ГАУЗ ТО «Городская поликлиника №17»; ГАУЗ ТО «Городская поликлиника №3»; ГБУЗ ТО "Областная больница № 11" (р.п. Голышманово); ГБУЗ ТО "Областная больница № 12" (г.Заводоуковск); ГБУЗ ТО "Областная больница № 14 им. В.Н.Шанаурина" (с. Казанское); ГБУЗ ТО "Областная больница № 15" (с.Нижняя Тавда); ГБУЗ ТО "Областная больница № 23" (г.Ялуторовск); ГБУЗ ТО "Областная больница № 24" (с.Ярково); ГБУЗ ТО "Областная больница № 3" (г.Тобольск); ГБУЗ ТО "Областная больница № 4" (г.Ишим); ГБУЗ ТО "Областная больница № 9" (с.Вагай); 

In [390]:
# Это дата выгрузки по МО из ФРМР
upload_date = '02.02.2024'

In [391]:
# Это дата выгрузки отчетов original и workers  по пятницам
period_last = '22.02.2024'

In [392]:
# Указываем верхнюю и нижнюю границу периода
start_date = '2024-02-19'
end_date = '2024-02-25'

In [393]:
period = f"{start_date} - {end_date}"

In [394]:
period

'2024-02-19 - 2024-02-25'

In [395]:
# Переменная окружения
ROOT_PATH = os.getcwd()
# поправочный файл от Анны Флягиной в котором учет должности ведется в перекрестной таблице
POST_AD = ROOT_PATH + '\\guide\\uchet_doljnosty.xlsx'
# в этой папке содержится отчет по штатным единицам из ФРМР по каждой МО они должны обновляться раз в неделю
FRMR_FILES = ROOT_PATH + '\\frmr'
# в этой папке содержится справочник МО
GUIDE_MEDORGANIZATION = ROOT_PATH + '\\guide\\medicine_organization_2.xlsx'
# в этой папке содержится список исключений которые мы не принимаем в расчет файл формирует Анна Флягина
DELETE_ORGANIZATION = ROOT_PATH + '\\guide\\ОБЩИЙ ДЕКАБРЬ.xlsx'
# Это отчет о сотрудниках работающих в организации который формируется в ФРМР на дату аналогичную выгрузке из АНАЛИТ (например на 8 число для 1 слайда) для второго слайда - на 4 число, это другая выгрузка
WORKERS_XLS = ROOT_PATH + '\\input\\workers.xlsx'
#Это перечень должностей которые мы учитываем при подсчете
LIST_DOLJNOST = ROOT_PATH + '\\guide\\tablica_doljnost.xlsx'
# Это данные из Аналит-расписание на пятницу
ORIGINAL_ANALIT = ROOT_PATH + '\\input\\original.xlsx'
# Это данные из Аналит расписания на понедельник
FILTR_ANALIT = ROOT_PATH + '\\input\\original_filtr.xlsx'
# Это перечень строк из ФРМР которые не следует использовать при расчетах
GUIDE_DELETE_MED = ROOT_PATH + '\\guide\\guide_delete.xlsx'
# Это норма слотов для каждой должности
NORMA_SLOTOV = ROOT_PATH + '\\guide\\norma_slotov.xlsx'
# Это записи на прием к врачу
RECORD_DOCTORS = ROOT_PATH + '\\input\\record.xlsx'
# Это неотложка
EMERGENCY_XLSX = ROOT_PATH + '\\emergency'
NEOTLOJKA_XLSX = ROOT_PATH + '\\emergency\\neotlojka.xlsx'
# Это три файла выгрузки для обработки аналит
ANALIT_SLIDE5 = ROOT_PATH + '\\analitSlide5'
# Это расписание 1С
RASPISANIE_1C =  ROOT_PATH + '\\input\\1c_raspisanie.xlsx'
# Это справочник должностей и соответствующих им специальностей из аналит
DOLJNOST_ANALIT = ROOT_PATH + '\\guide\\analit_doljnost.xlsx'
# Это корректировочный файл для удаленя из 1С
DELETE_RAS_1C = ROOT_PATH + '\\input\\Del_ras1c.xlsx'
# Это еще один корректировочный файл для файло которые мы не берем из original_filtr
DELETE_ORIGINAL_FILTR = ROOT_PATH + '\\guide\\analit_original_filter_delete.xlsx'
# Это порядок расположения столбцов
SEQUENCE = pd.read_excel(ROOT_PATH + '\\guide\\psequence.xlsx')

##### Порядок отображения МО

In [396]:
sequence_mo = SEQUENCE

##### Настройка цветов презентации

In [397]:
color_red = RGBColor(246, 175, 171)
color_green = RGBColor(153, 219, 161)
color_neutral = RGBColor(223, 220, 230)
color_header = RGBColor(99, 89, 122)
color_title = RGBColor(55, 34, 105)
color_yallow = RGBColor(246, 235, 171)
# Задание черного цвета границы таблицы
border_color = RGBColor(0, 0, 0)  # Черный цвет (RGB: 0, 0, 0)

##### Подготавливаем данные для слайда 1, столбца 1

In [398]:
# Импортируем справочник медорганизаций
medorganization = pd.read_excel(GUIDE_MEDORGANIZATION)

In [399]:

# Загружаем выгрузки из ФРМР по МО и объединяем
my_excel = glob.glob(os.path.join(FRMR_FILES,"*.xlsx"))
len(my_excel)
svod = []
for my_files in my_excel:
    next_df = pd.read_excel(my_files, sheet_name=1)
    next_df['МО'] = my_files.split("\\")[-1]
    svod.append(next_df)
frmr = pd.concat(svod, sort=False, axis=0)
frmr['МО'] = frmr['МО'].str.replace(r'.xlsx$', '', regex=True)
frmr = frmr[frmr['Должность'] != 'Итого:']
frmr = frmr.fillna(0)

In [400]:
frmr['Должность по Фед. справочнику'] = frmr['Должность по Фед. справочнику'].str.lower()

##### Фильтруем по должности

In [401]:
tablica_doljnost = pd.read_excel(LIST_DOLJNOST)
employee_list = tablica_doljnost['Должность'].to_list()
frmr_all = frmr[frmr['Должность'].isin(employee_list)] # Это фильтр по должности
frmr_all = frmr_all[frmr_all['Отделение стационара'] == 0] # Это исключение стационаров

In [402]:
# Это собранный Аней Флягиной список исключений
deletion = pd.read_excel(DELETE_ORGANIZATION, sheet_name=1, usecols=['Структурное подразделение', 'Должность', 'Должность по Фед. справочнику', 
                                                                     'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 
                                                                     'Вакансий', 'MO']) 
deletion.rename(columns={'MO': 'МО'}, inplace=True)
deletion = deletion[['МО', 'Структурное подразделение', 'Должность', 'Должность по Фед. справочнику']]

In [403]:
# Мержим обработанную выгрузку из ФРМР с исключениями и убираем исключения
frmr_all = frmr_all.merge(deletion, on=['Структурное подразделение', 'Должность', 'Должность по Фед. справочнику', 'МО'], how='left', indicator=True)
frmr_all = frmr_all.query('_merge == "left_only"').drop('_merge', axis=1)
frmr_all.reset_index(drop=True, inplace=True)

In [404]:
test_frmr_all_ftisio_unique = frmr_all[frmr_all['МО'] == 'ОКФЦ']

In [405]:
test_frmr_all_ftisio_unique

,Структурное подразделение,Отделение стационара,Должность,Должность по Фед. справочнику,Штатных единиц,Занятых штатных единиц,В т.ч. внешнее совместительство,Вакансий,Примечание,МО
605,Общебольничный медицинский персонал Тюмень,0,врач-оториноларинголог,врач-оториноларинголог,1.00,1.00,0.00,0.0,0,ОКФЦ
606,Общебольничный медицинский персонал Тюмень,0,врач-офтальмолог,врач-офтальмолог,1.00,1.00,0.00,0.0,0,ОКФЦ
607,Общебольничный медицинский персонал Тюмень,0,врач-стоматолог-терапевт,врач-стоматолог-терапевт,0.25,0.25,0.00,0.0,0,ОКФЦ
608,Общебольничный медицинский персонал Ишимского ...,0,врач-фтизиатр,врач-фтизиатр,0.25,0.25,0.00,0.0,0,ОКФЦ
609,Амбулаторное отделение Ишимского филиала,0,врач-фтизиатр участковый,врач-фтизиатр участковый,2.00,2.00,0.00,0.0,0,ОКФЦ
610,Общебольничный медицинский персонал Заводоуков...,0,врач-фтизиатр,врач-фтизиатр,0.50,0.50,0.00,0.0,0,ОКФЦ
611,Амбулаторное отделение Тобольского филиала,0,врач-фтизиатр участковый,врач-фтизиатр участковый,4.00,4.00,0.00,0.0,0,ОКФЦ
612,Амбулаторное отделение Тюмень,0,врач-фтизиатр участковый,врач-фтизиатр участковый,19.00,19.00,0.00,0.0,0,ОКФЦ
613,Амбулаторное отделение Тюмень,0,врач-фтизиатр,врач-фтизиатр,0.75,0.75,0.75,0.0,0,ОКФЦ
614,Общебольничный медицинский персонал Тюмень,0,врач-фтизиатр,врач-фтизиатр,9.75,9.75,0.75,0.0,0,ОКФЦ


##### Мержим обработанный список со справочником медорганизаций чтобы привести названия в нужный нам вид

In [406]:
frmr_all_merge = pd.merge(frmr_all, medorganization, on='МО', how='left')
frmr_all_merge = frmr_all_merge[['Организация', 'Структурное подразделение', 'Отделение стационара', 'Должность по Фед. справочнику', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 'Вакансий']]
frmr_all_merge.rename(columns={'Должность по Фед. справочнику': 'Должность'}, inplace=True)

In [407]:
frmr_all_merge['Организация'].unique()

array([nan, 'ГАУЗ ТО «ГП №1»', 'ГАУЗ ТО «ГП №12»', 'ГАУЗ ТО «ГП №13»',
       'ГАУЗ ТО «ГП №17»', 'ГАУЗ ТО «ГП №3»', 'ГАУЗ ТО «ГП №4»',
       'ГАУЗ ТО «ГП №5»', 'ГАУЗ ТО «ГП №6»', 'ГАУЗ ТО «ГП №8»',
       'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «ОБ №13»',
       'ГБУЗ ТО «ОБ №14»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №19»',
       'ГБУЗ ТО «ОБ №20»', 'ГБУЗ ТО «ОБ №23»', 'ГБУЗ ТО «ОБ №24»',
       'ГБУЗ ТО «ОБ №3»', 'ГБУЗ ТО «ОБ №4»', 'ГБУЗ ТО «ОБ №9»',
       'ГБУЗ ТО «ОКБ №1»', 'ГБУЗ ТО «ОКБ №2»', 'ГБУЗ ТО «ОКПБ»',
       'ГБУЗ ТО «ОКФЦ»', 'ГБУЗ ТО «ОНД»', 'ГАУЗ ТО "ОСП"',
       'ГБУЗ ТО «Перинатальный центр»', 'ГБУЗ ТО «Роддом № 2»',
       'ГБУЗ ТО «Родильный дом №3»'], dtype=object)

In [408]:
# Загрузка корректировочного файла от Анны Флягиной
uchet_doljnosty = pd.read_excel(POST_AD).fillna(0)
employee_uchet = pd.melt(uchet_doljnosty, id_vars=['Организация'], value_vars=employee_list, var_name='Должность', value_name='Проверка')
employee_uchet.sort_values(by='Проверка', ascending=True, inplace=True)

##### Объединяем с проверочным файлом в котором указаны должности по которым мы не считаем и удаляем ненужные

In [409]:
frmr_all_uchet = pd.merge(frmr_all_merge, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)
frmr_all_uchet = frmr_all_uchet[frmr_all_uchet['Проверка'] != 0]

In [410]:
frmr_all_uchet['Организация'].unique()

array(['ГАУЗ ТО «ГП №1»', 'ГАУЗ ТО «ГП №12»', 'ГАУЗ ТО «ГП №13»',
       'ГАУЗ ТО «ГП №17»', 'ГАУЗ ТО «ГП №3»', 'ГАУЗ ТО «ГП №4»',
       'ГАУЗ ТО «ГП №5»', 'ГАУЗ ТО «ГП №6»', 'ГАУЗ ТО «ГП №8»',
       'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «ОБ №13»',
       'ГБУЗ ТО «ОБ №14»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №19»',
       'ГБУЗ ТО «ОБ №20»', 'ГБУЗ ТО «ОБ №23»', 'ГБУЗ ТО «ОБ №24»',
       'ГБУЗ ТО «ОБ №3»', 'ГБУЗ ТО «ОБ №4»', 'ГБУЗ ТО «ОБ №9»',
       'ГБУЗ ТО «ОКБ №2»', 'ГБУЗ ТО «ОКПБ»', 'ГБУЗ ТО «ОКФЦ»',
       'ГБУЗ ТО «ОНД»', 'ГАУЗ ТО "ОСП"', 'ГБУЗ ТО «Роддом № 2»',
       'ГБУЗ ТО «Родильный дом №3»'], dtype=object)

In [411]:
proverka_all_uchet = frmr_all_uchet[frmr_all_uchet['Организация'] == 'ГБУЗ ТО «ОКФЦ»']

##### Расчет Слайд 1 Столбец 1

In [412]:
# формирование первого столбца первого слайда
frmr_all_slide_1 = frmr_all_uchet[['Организация', 'Должность']]

In [413]:
test_ftisiatr  = frmr_all_uchet[['Организация', 'Должность']]

In [414]:
test_ftisiatr['Организация'].unique()

array(['ГАУЗ ТО «ГП №1»', 'ГАУЗ ТО «ГП №12»', 'ГАУЗ ТО «ГП №13»',
       'ГАУЗ ТО «ГП №17»', 'ГАУЗ ТО «ГП №3»', 'ГАУЗ ТО «ГП №4»',
       'ГАУЗ ТО «ГП №5»', 'ГАУЗ ТО «ГП №6»', 'ГАУЗ ТО «ГП №8»',
       'ГБУЗ ТО «ОБ №11»', 'ГБУЗ ТО «ОБ №12»', 'ГБУЗ ТО «ОБ №13»',
       'ГБУЗ ТО «ОБ №14»', 'ГБУЗ ТО «ОБ №15»', 'ГБУЗ ТО «ОБ №19»',
       'ГБУЗ ТО «ОБ №20»', 'ГБУЗ ТО «ОБ №23»', 'ГБУЗ ТО «ОБ №24»',
       'ГБУЗ ТО «ОБ №3»', 'ГБУЗ ТО «ОБ №4»', 'ГБУЗ ТО «ОБ №9»',
       'ГБУЗ ТО «ОКБ №2»', 'ГБУЗ ТО «ОКПБ»', 'ГБУЗ ТО «ОКФЦ»',
       'ГБУЗ ТО «ОНД»', 'ГАУЗ ТО "ОСП"', 'ГБУЗ ТО «Роддом № 2»',
       'ГБУЗ ТО «Родильный дом №3»'], dtype=object)

In [415]:

frmr_all_slide_1.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1896369310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1.rename(columns={'Организация': 'Наименование МО'}, inplace=True)


In [416]:
frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-фтизиатр', 'врач-фтизиатр участковый', frmr_all_slide_1['Должность'])

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1793884492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-фтизиатр', 'врач-фтизиатр участковый', frmr_all_slide_1['Должность'])


In [417]:
frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-психиатр детский', 'врач-психиатр детский участковый', frmr_all_slide_1['Должность'])

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\2508088442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frmr_all_slide_1['Должность'] = np.where(frmr_all_slide_1['Должность'] == 'врач-психиатр детский', 'врач-психиатр детский участковый', frmr_all_slide_1['Должность'])


In [418]:
double_frmr_slide_1 = frmr_all_slide_1.drop_duplicates()

In [419]:
double_frmr_slide_1

,Наименование МО,Должность
4,ГАУЗ ТО «ГП №1»,врач-хирург
5,ГАУЗ ТО «ГП №1»,врач-оториноларинголог
6,ГАУЗ ТО «ГП №1»,врач - детский хирург
7,ГАУЗ ТО «ГП №1»,врач-офтальмолог
9,ГАУЗ ТО «ГП №1»,врач-терапевт участковый
...,...,...
615,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог
617,"ГАУЗ ТО ""ОСП""",врач-стоматолог-терапевт
619,"ГАУЗ ТО ""ОСП""",врач-стоматолог
632,ГБУЗ ТО «Роддом № 2»,врач-акушер-гинеколог


In [420]:
slide_1_column1 = pd.pivot_table(double_frmr_slide_1, 
                                 index='Наименование МО', 
                                 values='Должность', 
                                 aggfunc='count', 
                                 margins=True, 
                                 margins_name='Итого').reset_index()

In [421]:
slide_1_column1.rename(columns={'Должность': 'Кол-во должностей из 14'}, inplace=True)
slide_1_column1 = slide_1_column1[(slide_1_column1['Наименование МО'] != 'ГБУЗ ТО «ОКБ №1»') &   
                                  (slide_1_column1['Наименование МО'] != 'ГБУЗ ТО «Перинатальный центр»')]
slide_1_column1.reset_index(drop=True, inplace=True)

##### СЛАЙД 1 СТОЛБЕЦ 2

In [422]:
workers = pd.read_excel(WORKERS_XLS, skiprows=6)
workers.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)

c:\Users\propp_sa\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [423]:
workers['Должность по фед справочнику'] = workers['Должность по фед справочнику'].str.lower()

In [424]:
post_array = workers['Должность по фед справочнику'].unique()
post_df = pd.DataFrame({'Должность':post_array})

In [425]:
guide_post = pd.merge(tablica_doljnost, post_df, on='Должность', how='inner')
guide_post = guide_post.rename(columns={'Должность' : 'Должность по фед справочнику'})

In [426]:
# Оставляем только амбулаторные подразделения
workers = workers[workers['Тип структурного подразделения'] == 'Амбулаторный']
# Подгружаем справочник на удаление
delete_rows = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)
delete_rows = delete_rows[['Краткое наименование организации', 'OID структурного подразделения', 'Должность по фед справочнику']]
delete_rows['Удалить'] = 'True'
# Мержим со справочником на удаление и удаляем все значения не соответсвующие типу амбулаторный из исходного массива
result = pd.merge(workers, delete_rows,on=['Краткое наименование организации', 
                                      'OID структурного подразделения', 
                                      'Должность по фед справочнику'], how='left')
result = result[result['Удалить'].isnull()]


In [427]:
# Удаление дубликатов из первого DataFrame
guide_post['Delete'] = True
result_filtered = pd.merge(result, guide_post, on=['Должность по фед справочнику'], how='left')


In [428]:
result_filtered = result_filtered[result_filtered['Delete']  == True]

In [429]:
resultSlide = pd.merge(result_filtered, medorganization, on='Краткое наименование организации', how='left')

In [430]:
resultSlide.drop('Должность', axis=1, inplace=True)

In [431]:
resultSlide.rename(columns={'Должность по фед справочнику': 'Должность'}, inplace=True)

In [432]:
itog1 = pd.pivot_table(resultSlide, 
                      index=['Организация', 'Должность'], 
                      columns=['Причина временного неисполнения функциональных обязанностей'], 
                      values='СНИЛС', 
                      aggfunc=pd.Series.nunique).fillna(0)
itog1.reset_index(inplace=True)

In [433]:
itog1_slide1 = pd.merge(itog1, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [434]:
itog1_slide1 = itog1_slide1[itog1_slide1['Проверка'] != 0]

In [435]:
itog2 = pd.pivot_table(resultSlide, 
                      index=['Организация', 'Должность'], 
                      values='СНИЛС', 
                      aggfunc=pd.Series.nunique)
itog2.reset_index(inplace=True)

In [436]:
itog2_slide1 = pd.merge(itog2, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [437]:
itog2_slide1[itog2_slide1['Организация'] == 'ГБУЗ ТО «ОНД»']

,Организация,Должность,СНИЛС,Проверка
245,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог,13,1.0


In [438]:
itog = pd.merge(itog2_slide1, itog1_slide1, on=['Организация', 'Должность', 'Проверка'], how='outer').fillna(0)

In [439]:
guide_post2 = pd.merge(tablica_doljnost, post_df, on='Должность', how='inner')

In [440]:
itog_merge = pd.merge(guide_post2, itog, on=['Должность'], how='left')

In [441]:
itog_merge = itog_merge[itog_merge['Проверка'] != 0]

In [442]:
itog_merge_mo = pd.merge(medorganization, itog_merge, on='Организация', how='left').dropna()

In [443]:
itog_merge_mo = itog_merge_mo[['Организация', 'Должность', 'СНИЛС', 'временная нетрудоспособность', 
                               'дополнительное профессиональное образование', 'иные причины', 'отпуск без сохранения заработной платы', 
                               'отпуск по беременности и родам', 'отпуск по уходу за ребенком']]

In [444]:
itog_merge_mo[itog_merge_mo['Организация'] == 'ГБУЗ ТО «ОКПБ»']

,Организация,Должность,СНИЛС,временная нетрудоспособность,дополнительное профессиональное образование,иные причины,отпуск без сохранения заработной платы,отпуск по беременности и родам,отпуск по уходу за ребенком
220,ГБУЗ ТО «ОКПБ»,врач-психиатр детский участковый,7.0,0.0,0.0,0.0,0.0,0.0,1.0


In [445]:
res = itog_merge_mo.sort_values(by=['Организация', 'Должность']).reset_index(drop=True)

In [446]:
res.rename(columns={'СНИЛС' : 'Всего'}, inplace=True)

In [447]:
res['Итого'] = res['Всего'] - res['временная нетрудоспособность'] - res['дополнительное профессиональное образование'] - res['иные причины'] - res['отпуск без сохранения заработной платы'] - res['отпуск по беременности и родам'] - res['отпуск по уходу за ребенком']
slide_1_column2 = pd.pivot_table(res, 
                                 index='Организация', 
                                 values='Всего', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()

In [448]:
slide_1_column2 = pd.merge(medorganization, slide_1_column2, on='Организация', how='outer')

In [449]:
slide_1_column2 = slide_1_column2[(slide_1_column2['Краткое наименование организации'] != 'Тюменская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'Тобольская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО "ЦПБС"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО «Областной лечебно-реабилитационный центр»') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО «Госпиталь для ветеранов войн»') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "ОБЛАСТНОЙ ОФТАЛЬМОЛОГИЧЕСКИЙ ДИСПАНСЕР"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "МКМЦ "МЕДИЦИНСКИЙ ГОРОД"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "МКДЦ"') & 
                                  (slide_1_column2['Краткое наименование организации'] != 'ГАУЗ ТО "ДЛРЦ "НАДЕЖДА"') &
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО «ОКБ № 1»') &
                                  (slide_1_column2['Краткое наименование организации'] != 'АО "МСЧ "НЕФТЯНИК"') &
                                  (slide_1_column2['Краткое наименование организации'] != 'ГБУЗ ТО "ПЕРИНАТАЛЬНЫЙ ЦЕНТР" (Г.ТЮМЕНЬ)')]

In [450]:
slide_1_column2 = slide_1_column2[['Организация', 'Всего']]
slide_1_column2.reset_index(drop=True, inplace=True)
slide_1_column2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_1_column2['Наименование МО'] = slide_1_column2['Наименование МО'].fillna('Итого')

##### Слайд 1 Столбец 3

In [451]:
slide_1_column3 = pd.pivot_table(res, 
                                 index='Организация', 
                                 values='Итого', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()
slide_1_column3 = pd.merge(medorganization, slide_1_column3, on='Организация', how='outer')
slide_1_column3 = slide_1_column3[(slide_1_column3['Краткое наименование организации'] != 'Тюменская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'Тобольская больница ФГБУЗ ЗСМЦ ФМБА России') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО "ЦПБС"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО «Областной лечебно-реабилитационный центр»') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО «Госпиталь для ветеранов войн»') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "ОБЛАСТНОЙ ОФТАЛЬМОЛОГИЧЕСКИЙ ДИСПАНСЕР"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "МКМЦ "МЕДИЦИНСКИЙ ГОРОД"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "МКДЦ"') & 
                                  (slide_1_column3['Краткое наименование организации'] != 'ГАУЗ ТО "ДЛРЦ "НАДЕЖДА"') &
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО «ОКБ № 1»') &
                                  (slide_1_column3['Краткое наименование организации'] != 'АО "МСЧ "НЕФТЯНИК"') &
                                  (slide_1_column3['Краткое наименование организации'] != 'ГБУЗ ТО "ПЕРИНАТАЛЬНЫЙ ЦЕНТР" (Г.ТЮМЕНЬ)')]
slide_1_column3 = slide_1_column3[['Организация', 'Итого']]
slide_1_column3.reset_index(drop=True, inplace=True)
slide_1_column3.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_1_column3['Наименование МО'] = slide_1_column3['Наименование МО'].fillna('Итого')

###### Создаем слайд 1 столбец 4
* Файл original выгружается из аналит расписания за период в одну неделю вперед по пути Статистика Здравоохранение - Организация работы фронт-офиса - Деталзация расписания врачей.

In [452]:
original = pd.read_excel(ORIGINAL_ANALIT)

In [453]:
original = original[(original['Медицинская организация'] != 'Всего')]
analit_merge = pd.merge(original, medorganization, on='Медицинская организация', how='left')
analit_result  = analit_merge[['Медицинская организация', 'Краткое наименование организации', 'Отделение', 'Отделение (адрес)',
       'Должность', 'Специальность', 'Врач', 'Дней расписания', 'Слотов на 7 дней', 'Слотов на 14 дней', 'Слотов на 30 дней',
       'Формиров. расписания на 7 дней, %', 'Формиров. расписания на 14 дней, %', 'Формиров. расписания на 30 дней, %',
       'Слотов первичного приёма на 7 дней', 'Доля первичного приёма на 7 дней, %', 'Слотов первичного приёма на 14 дней',
       'Доля первичного приёма на 14 дней, %', 'Слотов первичного приёма на 30 дней', 'Доля первичного приёма на 30 дней, %']]
delete_rows2 = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows2.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)
delete_rows2 = delete_rows2[['Краткое наименование организации', 'Наименование структурного подразделения', 'Должность по фед справочнику']]
delete_rows2 = delete_rows2.rename(columns={'Должность по фед справочнику' : 'Должность', 'Наименование структурного подразделения': 'Отделение'})
delete_rows2['Удалить'] = 'True'
result_original = pd.merge(analit_result, delete_rows2,on=['Краткое наименование организации', 
                                      'Отделение', 
                                      'Должность'], how='left')
result_original = result_original[result_original['Удалить'] != 'True']

In [454]:
result_original

,Медицинская организация,Краткое наименование организации,Отделение,Отделение (адрес),Должность,Специальность,Врач,Дней расписания,Слотов на 7 дней,Слотов на 14 дней,...,"Формиров. расписания на 7 дней, %","Формиров. расписания на 14 дней, %","Формиров. расписания на 30 дней, %",Слотов первичного приёма на 7 дней,"Доля первичного приёма на 7 дней, %",Слотов первичного приёма на 14 дней,"Доля первичного приёма на 14 дней, %",Слотов первичного приёма на 30 дней,"Доля первичного приёма на 30 дней, %",Удалить
0,ГАУЗ ТО «Госпиталь для ветеранов войн»,NaN,Гериатрическое отделение,"625023, Тюменская обл, г Тюмень, ул Котовского...",врач-гериатр,Лечебное дело,Плановая госпитализация ГЕРИАТРИЯ,9,32,76,...,1.000000,0.642857,0.300000,0,0,0,0,0,0.0,NaN
1,ГАУЗ ТО «Госпиталь для ветеранов войн»,NaN,Дневной стационар,"625000, Тюменская обл, Тюмень г, Володарского ...",врач-терапевт,Терапия,Сагадиева Марина Сергеевна ДС,21,40,80,...,1.000000,1.000000,0.700000,0,0,0,0,0,0.0,NaN
2,ГАУЗ ТО «Госпиталь для ветеранов войн»,NaN,Поликлиника,ул.Володарского 47/1,врач ультразвуковой диагностики,Ультразвуковая диагностика,Балдина Наталья Павловна,15,42,95,...,1.000000,1.000000,0.500000,0,0,0,0,0,0.0,NaN
3,ГАУЗ ТО «Госпиталь для ветеранов войн»,NaN,Поликлиника,ул.Володарского 47/1,врач ультразвуковой диагностики,Ультразвуковая диагностика,Курдоглян Осанна Сергеевна УЗИ,16,132,264,...,1.000000,1.000000,0.533333,0,0,0,0,0,0.0,NaN
4,ГАУЗ ТО «Госпиталь для ветеранов войн»,NaN,Поликлиника,ул.Володарского 47/1,врач функциональной диагностики,Функциональная диагностика,Петренко Оксана Алексеевна,11,105,210,...,1.000000,0.785714,0.366667,0,0,0,0,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,Поликлиника,"625015, Тюменская обл, г Тюмень, ул Беляева, д. 1",заведующий отделом медицинской организации,Терапия,Васютина Мария Александровна,20,132,297,...,1.000000,1.000000,0.666667,0,0,0,0,0,0.0,NaN
5623,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,Поликлиника,"625015, Тюменская обл, г Тюмень, ул Беляева, д. 1",медицинский психолог,Психотерапия,Гаджиалиева Айна Назировна,4,14,28,...,0.571429,0.285714,0.133333,0,0,0,0,0,0.0,NaN
5624,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,Поликлиника,"625015, Тюменская обл, г Тюмень, ул Беляева, д. 1",фельдшер,Лечебное дело,Азисова Сания Мавлитзяновна,30,252,504,...,1.000000,1.000000,1.000000,0,0,0,0,0,0.0,NaN
5625,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,Поликлиника,"625015, Тюменская обл, г Тюмень, ул Беляева, д. 1",фельдшер,Лечебное дело,Плотникова Анастасия Александровна,20,120,270,...,1.000000,1.000000,0.666667,0,0,0,0,0,0.0,NaN


###### Порядок работы с данными
* Вначале смержить со справочником медорганизаций
* потом смержить с таблицей исключений
* Потом убрать исключения
* Только затем мержить с остальными данными

In [455]:
resultSlide1_4 = pd.merge(result_original, medorganization, on=['Медицинская организация', 'Краткое наименование организации'], how='outer')[[ 'Организация', 'Отделение', 'Отделение (адрес)', 'Должность', 'Специальность', 'Врач',
'Дней расписания', 'Слотов на 7 дней', 'Слотов на 14 дней',
'Слотов на 30 дней', 'Формиров. расписания на 7 дней, %',
'Формиров. расписания на 14 дней, %',
'Формиров. расписания на 30 дней, %',
'Слотов первичного приёма на 7 дней',
'Доля первичного приёма на 7 дней, %',
'Слотов первичного приёма на 14 дней',
'Доля первичного приёма на 14 дней, %',
'Слотов первичного приёма на 30 дней',
'Доля первичного приёма на 30 дней, %', 'Удалить']]

In [456]:
resultSlide1_4 = pd.merge(resultSlide1_4, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [457]:
resultSlide1_4 = resultSlide1_4[resultSlide1_4['Проверка'] != 0]

In [458]:
table_analit_doljnost = pd.read_excel(DOLJNOST_ANALIT)
merge = pd.merge(resultSlide1_4, table_analit_doljnost, on = ['Должность', 'Специальность'], how='right')

In [459]:
df1 = merge
df2 = merge
df1['Доля первичного приёма на 7 дней, %'] = df1['Доля первичного приёма на 7 дней, %'].replace('-', 0)
df1['Доля первичного приёма на 14 дней, %'] = df1['Доля первичного приёма на 14 дней, %'].replace('-', 0)
df1['Доля первичного приёма на 30 дней, %'] = df1['Доля первичного приёма на 30 дней, %'].replace('-', 0)
svod_slotov = pd.pivot_table(df1, values=['Слотов на 7 дней', 
                                          'Слотов на 14 дней', 
                                          'Слотов на 30 дней', 
                                          'Слотов первичного приёма на 7 дней', 
                                          'Слотов первичного приёма на 14 дней', 
                                          'Слотов первичного приёма на 30 дней'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=np.sum, 
                             fill_value=0)
svod_slotov.reset_index(drop=False, inplace=True)
def proz7days(row):
    if row['Слотов первичного приёма на 7 дней'] != 0 and row['Слотов на 7 дней'] != 0:
        val = round(row['Слотов первичного приёма на 7 дней'] / row['Слотов на 7 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 7 дней'] = svod_slotov.apply(proz7days, axis = 1)
def proz14days(row):
    if row['Слотов первичного приёма на 14 дней'] != 0 and row['Слотов на 14 дней'] != 0:
        val = round(row['Слотов первичного приёма на 14 дней'] / row['Слотов на 14 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 14 дней'] = svod_slotov.apply(proz14days, axis = 1)
def proz30days(row):
    if row['Слотов первичного приёма на 30 дней'] != 0 and row['Слотов на 30 дней'] != 0:
        val = round(row['Слотов первичного приёма на 30 дней'] / row['Слотов на 30 дней'] * 100)
    else:
        val = 0
    return val
svod_slotov['% конкурентных слотов на 30 дней'] = svod_slotov.apply(proz30days, axis = 1)
df2_7days = df2[df2['Доля первичного приёма на 7 дней, %'] != '-']
df2_14days = df2[df2['Доля первичного приёма на 14 дней, %'] != '-']
df2_30days = df2[df2['Доля первичного приёма на 30 дней, %'] != '-']
unigue_7days = pd.pivot_table(df2_7days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_14days = pd.pivot_table(df2_14days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_30days = pd.pivot_table(df2_30days, values=['Врач'], 
                             index=['Организация', 'Должность'], 
                             aggfunc=pd.Series.nunique, 
                             fill_value=0)
unigue_7days.reset_index(drop=False, inplace=True)
unigue_14days.reset_index(drop=False, inplace=True)
unigue_30days.reset_index(drop=False, inplace=True)
unigue_7days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 7 дней'}, inplace=True)
unigue_14days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 14 дней'}, inplace=True)
unigue_30days.rename(columns = {'Врач': 'кол-во сотрудников с сформированным расписанием на 30 дней'}, inplace=True)
all_days = [svod_slotov, unigue_7days, unigue_14days, unigue_30days]
itog = reduce(lambda left,right: pd.merge(left,right,on=['Организация', 'Должность'], how='outer'), all_days).fillna('none')

In [460]:
employee_with_schedule =itog.copy()

In [461]:
employee_with_schedule

,Организация,Должность,Слотов на 14 дней,Слотов на 30 дней,Слотов на 7 дней,Слотов первичного приёма на 14 дней,Слотов первичного приёма на 30 дней,Слотов первичного приёма на 7 дней,% конкурентных слотов на 7 дней,% конкурентных слотов на 14 дней,% конкурентных слотов на 30 дней,кол-во сотрудников с сформированным расписанием на 7 дней,кол-во сотрудников с сформированным расписанием на 14 дней,кол-во сотрудников с сформированным расписанием на 30 дней
0,"ГАУЗ ТО ""ОСП""",врач-стоматолог,2739,2939,1398,1722,1842,919,66,63,63,29,29,29
1,"ГАУЗ ТО ""ОСП""",врач-стоматолог-терапевт,2292,2504,1000,1430,1563,623,62,62,62,26,26,26
2,ГАУЗ ТО «ГП №12»,врач - детский хирург,361,750,157,253,536,103,66,70,71,3,3,3
3,ГАУЗ ТО «ГП №12»,врач-акушер-гинеколог,1624,3377,757,1137,2409,530,70,70,71,15,15,15
4,ГАУЗ ТО «ГП №12»,врач-оториноларинголог,1363,2895,639,959,2046,452,71,70,71,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,ГБУЗ ТО «ОКПБ»,врач-психиатр детский участковый,809,1122,330,554,785,225,68,68,70,7,7,7
214,ГБУЗ ТО «ОКФЦ»,врач-фтизиатр участковый,2810,3760,1356,1892,2541,915,67,67,68,25,25,25
215,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог,1351,2819,633,897,1863,424,67,66,66,12,12,12
216,ГБУЗ ТО «Роддом № 2»,врач-акушер-гинеколог,2326,5227,1014,1680,3738,743,73,72,72,10,10,10


In [462]:
employee_with_schedule.columns

Index(['Организация', 'Должность', 'Слотов на 14 дней', 'Слотов на 30 дней',
       'Слотов на 7 дней', 'Слотов первичного приёма на 14 дней',
       'Слотов первичного приёма на 30 дней',
       'Слотов первичного приёма на 7 дней', '% конкурентных слотов на 7 дней',
       '% конкурентных слотов на 14 дней', '% конкурентных слотов на 30 дней',
       'кол-во сотрудников с сформированным расписанием на 7 дней',
       'кол-во сотрудников с сформированным расписанием на 14 дней',
       'кол-во сотрудников с сформированным расписанием на 30 дней'],
      dtype='object')

In [463]:
employee_with_schedule = employee_with_schedule[['Организация', 'Должность', 'кол-во сотрудников с сформированным расписанием на 7 дней', 'Слотов на 7 дней', 'Слотов первичного приёма на 7 дней']]

In [464]:
employee_with_schedule

,Организация,Должность,кол-во сотрудников с сформированным расписанием на 7 дней,Слотов на 7 дней,Слотов первичного приёма на 7 дней
0,"ГАУЗ ТО ""ОСП""",врач-стоматолог,29,1398,919
1,"ГАУЗ ТО ""ОСП""",врач-стоматолог-терапевт,26,1000,623
2,ГАУЗ ТО «ГП №12»,врач - детский хирург,3,157,103
3,ГАУЗ ТО «ГП №12»,врач-акушер-гинеколог,15,757,530
4,ГАУЗ ТО «ГП №12»,врач-оториноларинголог,8,639,452
...,...,...,...,...,...
213,ГБУЗ ТО «ОКПБ»,врач-психиатр детский участковый,7,330,225
214,ГБУЗ ТО «ОКФЦ»,врач-фтизиатр участковый,25,1356,915
215,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог,12,633,424
216,ГБУЗ ТО «Роддом № 2»,врач-акушер-гинеколог,10,1014,743


In [80]:
itog = itog[['Организация', 'кол-во сотрудников с сформированным расписанием на 7 дней']]
itog.rename(columns={'кол-во сотрудников с сформированным расписанием на 7 дней': 'Кол-во физ-лиц на которые заведено расписание'}, inplace=True)

In [81]:
slide1_column4 = pd.pivot_table(itog, index='Организация', values=['Кол-во физ-лиц на которые заведено расписание'], aggfunc=np.sum).reset_index()
slide1_column4.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

In [82]:
# Добавление строки "Итого" с числовым индексом
total_row = pd.Series(['Итого', slide1_column4['Кол-во физ-лиц на которые заведено расписание'].sum()], index=slide1_column4.columns)
slide1_column4 = slide1_column4.append(total_row, ignore_index=True)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1673956601.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  slide1_column4 = slide1_column4.append(total_row, ignore_index=True)


##### Создаем Слайд 1 столбец 6 Здесь мы считаем тех кто в отпуске по беременности и родам

In [83]:
decree = res[['Организация', 'отпуск по беременности и родам', 'отпуск по уходу за ребенком']]

In [84]:
decree['Кол-во физ-лиц в ФРМР в ДО'] = decree['отпуск по беременности и родам'] + decree['отпуск по уходу за ребенком']

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1532569484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decree['Кол-во физ-лиц в ФРМР в ДО'] = decree['отпуск по беременности и родам'] + decree['отпуск по уходу за ребенком']


In [85]:
slide_1_column6 = pd.pivot_table(decree, 
                                 index='Организация', 
                                 values='Кол-во физ-лиц в ФРМР в ДО', 
                                 aggfunc=np.sum, 
                                 margins=True, 
                                 margins_name='Итого').reset_index()
slide_1_column6.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

##### Считаем 7 столбец 1 слайда

In [86]:
frmr_all_uchet

,Организация,Структурное подразделение,Отделение стационара,Должность,Штатных единиц,Занятых штатных единиц,В т.ч. внешнее совместительство,Вакансий,Проверка
4,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение (улиц...,0,врач-хирург,2.00,2.00,0.0,0.00,1.0
5,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач-оториноларинголог,1.00,1.00,0.0,0.00,1.0
6,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач - детский хирург,1.25,1.25,0.0,0.00,1.0
7,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач-офтальмолог,2.00,2.00,0.0,0.00,1.0
8,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение (улиц...,0,врач-оториноларинголог,2.00,2.00,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...,...
623,"ГАУЗ ТО ""ОСП""",Лечебно-профилактическое отделение № 1,0,врач-стоматолог-терапевт,14.75,14.75,0.0,0.00,1.0
624,"ГАУЗ ТО ""ОСП""",Стоматологическое отделение №7,0,врач-стоматолог,8.00,8.00,0.0,0.00,1.0
625,"ГАУЗ ТО ""ОСП""",Стоматологическое отделение №7,0,врач-стоматолог-терапевт,3.00,3.00,0.0,0.00,1.0
632,ГБУЗ ТО «Роддом № 2»,Женская консультация №2,0,врач-акушер-гинеколог,10.00,10.00,0.0,0.00,1.0


In [87]:
slide_1_column7 = pd.pivot_table(frmr_all_uchet, index='Организация', values='Занятых штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого').round(2).reset_index()

In [88]:
slide_1_column7.rename(columns={'Организация': 'Наименование МО', 'Занятых штатных единиц': 'Кол-во ставок в ШР МО'}, inplace=True)

In [89]:
slide_1_column7

,Наименование МО,Кол-во ставок в ШР МО
0,"ГАУЗ ТО ""ОСП""",54.25
1,ГАУЗ ТО «ГП №12»,147.00
2,ГАУЗ ТО «ГП №13»,43.25
3,ГАУЗ ТО «ГП №17»,107.00
4,ГАУЗ ТО «ГП №1»,66.25
5,ГАУЗ ТО «ГП №3»,92.75
6,ГАУЗ ТО «ГП №4»,64.00
7,ГАУЗ ТО «ГП №5»,169.00
8,ГАУЗ ТО «ГП №6»,57.00
9,ГАУЗ ТО «ГП №8»,84.25


In [90]:
slide_1_column7['Кол-во ставок в ШР МО'] = slide_1_column7['Кол-во ставок в ШР МО'].round(2)

#### Подготовка данных для слайда 2

1. Рассчитать долю фактического количества слотов от расчетных %
2. Рассчитать долю фактического количества слотов от расчетных у врачей участковой службы, %
3. Рассчитать долю конкурентных слотов от  общего количества слотов, %
4. Рассчитать доля конкурентных слотов участковой службы от общего количества слотов, %
5. Рассчитать итоги для каждого столбца с долей
6. Объединить в один датафрейм
7. Датафрейм использовать для построения таблицы и добавить таблицу на слайд
8. Настроить форматирование по условиям в таблице
9. Настроить оформление слайда

In [91]:
frmr_all_uchet

,Организация,Структурное подразделение,Отделение стационара,Должность,Штатных единиц,Занятых штатных единиц,В т.ч. внешнее совместительство,Вакансий,Проверка
4,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение (улиц...,0,врач-хирург,2.00,2.00,0.0,0.00,1.0
5,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач-оториноларинголог,1.00,1.00,0.0,0.00,1.0
6,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач - детский хирург,1.25,1.25,0.0,0.00,1.0
7,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение для д...,0,врач-офтальмолог,2.00,2.00,0.0,0.00,1.0
8,ГАУЗ ТО «ГП №1»,Консультативно-диагностическое отделение (улиц...,0,врач-оториноларинголог,2.00,2.00,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...,...
623,"ГАУЗ ТО ""ОСП""",Лечебно-профилактическое отделение № 1,0,врач-стоматолог-терапевт,14.75,14.75,0.0,0.00,1.0
624,"ГАУЗ ТО ""ОСП""",Стоматологическое отделение №7,0,врач-стоматолог,8.00,8.00,0.0,0.00,1.0
625,"ГАУЗ ТО ""ОСП""",Стоматологическое отделение №7,0,врач-стоматолог-терапевт,3.00,3.00,0.0,0.00,1.0
632,ГБУЗ ТО «Роддом № 2»,Женская консультация №2,0,врач-акушер-гинеколог,10.00,10.00,0.0,0.00,1.0


In [92]:
frmr_all_slide_2 = frmr_all_uchet
frmr_all_slide_2 = frmr_all_slide_2[['Организация', 'Структурное подразделение', 'Отделение стационара', 'Должность', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство']]
norma = pd.read_excel(NORMA_SLOTOV)
frmr_all_slide_2 = pd.merge(frmr_all_uchet, norma, how='left', on='Должность')
frmr_all_slide_2 = frmr_all_slide_2[frmr_all_slide_2['Отделение стационара'] == 0]
frmr_all_slide_2['Сумма слотов на врача'] = frmr_all_slide_2['Занятых штатных единиц'] * frmr_all_slide_2['Норма слотов']
pivot_table_frmr = pd.pivot_table(frmr_all_slide_2, index='Организация', values='Сумма слотов на врача', aggfunc=np.sum, margins=True, margins_name='Итого')
pivot_table_frmr = pivot_table_frmr.reset_index()

#### Обрабатываю данные показателей записи на прием к врачу

In [93]:
indi = pd.read_excel(RECORD_DOCTORS, skiprows=[0, 1, 2, 3, 4])


In [94]:
indi

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Общее число слотов в расписании,из них конкурентных слотов,Общее число записей,"из них вне слота с признаком ""живая очередь""",из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени,...,Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний повторная,Число поступивших обращений с целью записи на прием к врачу-специалисту в рамках диспансерного наблюдения,Число поступивших обращений с целью записи на услуги инструментальной и/или лабораторной диагностики,"Число поступивших обращений с целью получения назначения, направления, справки, выписки, листа нетрудоспособности, иных документов",Число поступивших обращений с целью записи для прохождения первого или второго этапа диспансеризации,Число поступивших обращений с целью записи на вакцинацию,Число операторов колл-центров,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Число звонков для обратной связи (уточнение цели обращения и актуализация записи)
0,"АСУСОНТО ""Щучинский психоневрологический интер...",NaN,NaN,врач-терапевт,NaN,NaN,NaN,27.0,27.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"АСУСОНТО ""Щучинский психоневрологический интер...",NaN,NaN,фельдшер,NaN,NaN,NaN,62.0,62.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач - детский хирург,NaN,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,4.0,NaN
3,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач общей практики (семейный врач),NaN,138.0,77.0,174.0,NaN,61.0,...,88.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ГАУЗ ТО «ГП № 1»,NaN,NaN,врач по медицинской профилактике,NaN,794.0,NaN,706.0,9.0,87.0,...,163.0,81.0,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-терапевт,NaN,292.0,NaN,116.0,11.0,NaN,...,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN
425,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-уролог,NaN,34.0,20.0,103.0,63.0,7.0,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,врач-хирург,NaN,149.0,54.0,127.0,2.0,28.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,NaN,NaN,фельдшер,NaN,487.0,NaN,104.0,23.0,NaN,...,NaN,NaN,NaN,52.0,NaN,8.0,NaN,NaN,NaN,NaN


In [95]:
indicators = indi

In [96]:

indicators.rename(columns={'Unnamed: 0': 'МО', 'Unnamed: 3': 'Должность'}, inplace=True)
indicators = indicators[['МО', 'Должность', 'Общее число слотов в расписании']].fillna(0)
indicators.rename(columns={'МО': 'Медицинская организация'}, inplace=True)

In [97]:
indicators

,Медицинская организация,Должность,Общее число слотов в расписании
0,"АСУСОНТО ""Щучинский психоневрологический интер...",врач-терапевт,0.0
1,"АСУСОНТО ""Щучинский психоневрологический интер...",фельдшер,0.0
2,ГАУЗ ТО «ГП № 1»,врач - детский хирург,5.0
3,ГАУЗ ТО «ГП № 1»,врач общей практики (семейный врач),138.0
4,ГАУЗ ТО «ГП № 1»,врач по медицинской профилактике,794.0
...,...,...,...
424,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,врач-терапевт,292.0
425,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,врач-уролог,34.0
426,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,врач-хирург,149.0
427,Тюменская больница ФГБУЗ ЗСМЦ ФМБА,фельдшер,487.0


In [98]:
indicators['Медицинская организация'].unique()

array(['АСУСОНТО "Щучинский психоневрологический интернат"',
       'ГАУЗ ТО «ГП № 1»', 'ГАУЗ ТО «ГП № 12»', 'ГАУЗ ТО «ГП № 13»',
       'ГАУЗ ТО «ГП №4»', 'ГАУЗ ТО «ГП № 5»',
       'ГАУЗ ТО «Госпиталь для ветеранов войн»', 'ГАУЗ ТО «ГП14»',
       'ГАУЗ ТО «ДЛРЦ «Надежда»', 'ГАУЗ ТО «МКМЦ "Медицинский город»',
       'ГАУЗ ТО «МКДЦ»', 'ГАУЗ ТО «Областная больница № 19»',
       'ГАУЗ ТО «Областной офтальмологический диспансер»',
       'ГАУЗ ТО «ГП № 6»', 'ГАУЗ ТО «ГП № 8»', 'ГАУЗ ТО «ГП № 17»',
       'ГАУЗ ТО «ГП № 3»',
       'ГАУЗ ТО «Областной лечебно-реабилитационный центр»',
       'ГБУЗ ТО "Областная больница № 11" (р.п.Голышманово)',
       'ГБУЗ ТО "ОБ № 12"', 'ГБУЗ ТО ОБ №14', 'ГБУЗ ТО "ОБ № 15"',
       'ГБУЗ ТО "ОБ № 23"', 'ГБУЗ ТО «Областная больница № 24»',
       'ГБУЗ ТО "ОБ № 3"', 'ГБУЗ ТО "ОБ №4" (г.Ишим)', 'ГБУЗ ТО "ОБ № 9"',
       'ГБУЗ ТО "ОБ № 13"',
       'ГАУЗ ТО «Областная стоматологическая поликлиника»',
       'ГБУЗ ТО "ОКБ №1', 'ГБУЗ ТО "ОКБ №2"',
      

In [99]:
indicators[indicators['Медицинская организация'] == 'ГБУЗ ТО "ОБ №4" (г.Ишим)']

,Медицинская организация,Должность,Общее число слотов в расписании
301,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",0,0.0
302,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-терапевт участковый,4332.0
303,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-педиатр участковый,2434.0
304,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-терапевт,306.0
305,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",зубной врач,755.0
306,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-акушер-гинеколог,758.0
307,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач - детский хирург,48.0
308,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-оториноларинголог,454.0
309,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-офтальмолог,618.0
310,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",врач-педиатр,91.0


In [100]:
indicator_mo = pd.merge(indicators, medorganization, how='left', on='Медицинская организация').dropna()

In [101]:
indicator_mo = indicator_mo[['Организация', 'Должность', 'Общее число слотов в расписании']]

In [102]:
indicator_mo = indicator_mo[(indicator_mo['Должность'] == 'врач - детский хирург') | 
            (indicator_mo['Должность'] == 'врач общей практики (семейный врач)') |
            (indicator_mo['Должность'] ==  'врач-акушер-гинеколог') |
            (indicator_mo['Должность'] ==  'врач-оториноларинголог') |
            (indicator_mo['Должность'] ==  'врач-офтальмолог') |
            (indicator_mo['Должность'] ==  'врач-педиатр участковый') |
            (indicator_mo['Должность'] ==  'врач-психиатр детский') |
            (indicator_mo['Должность'] ==  'врач-психиатр-нарколог') |
            (indicator_mo['Должность'] ==  'врач-стоматолог') |
            (indicator_mo['Должность'] ==  'врач-стоматолог детский') |
            (indicator_mo['Должность'] ==  'врач-стоматолог-терапевт') |
            (indicator_mo['Должность'] ==  'врач-терапевт участковый') |
            (indicator_mo['Должность'] ==  'врач-фтизиатр') |
            (indicators['Должность'] ==  'врач-фтизиатр участковый') |
            (indicator_mo['Должность'] ==  'врач-хирург') |
            (indicator_mo['Должность'] ==  'врач-психиатр детский участковый')]

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\193165115.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indicator_mo = indicator_mo[(indicator_mo['Должность'] == 'врач - детский хирург') |


In [103]:
indicator_mo[indicator_mo['Организация'] == 'ГБУЗ ТО «ОБ №4»']

,Организация,Должность,Общее число слотов в расписании
302,ГБУЗ ТО «ОБ №4»,врач-терапевт участковый,4332.0
303,ГБУЗ ТО «ОБ №4»,врач-педиатр участковый,2434.0
306,ГБУЗ ТО «ОБ №4»,врач-акушер-гинеколог,758.0
307,ГБУЗ ТО «ОБ №4»,врач - детский хирург,48.0
308,ГБУЗ ТО «ОБ №4»,врач-оториноларинголог,454.0
309,ГБУЗ ТО «ОБ №4»,врач-офтальмолог,618.0
311,ГБУЗ ТО «ОБ №4»,врач-психиатр-нарколог,15.0
312,ГБУЗ ТО «ОБ №4»,врач-стоматолог,741.0
313,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,71.0
314,ГБУЗ ТО «ОБ №4»,врач-стоматолог-терапевт,382.0


##### Проверка с таблицей с единичками по должностям, исключение из расчета должности которые считать не следует

In [104]:
indicator_mo_uchet = pd.merge(indicator_mo, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [105]:
indicator_mo_uchet = indicator_mo_uchet[indicator_mo_uchet['Проверка'] != 0]

In [106]:
indicator_mo_uchet

,Организация,Должность,Общее число слотов в расписании,Проверка
0,ГАУЗ ТО «ГП №1»,врач - детский хирург,5.0,1.0
1,ГАУЗ ТО «ГП №1»,врач общей практики (семейный врач),138.0,1.0
2,ГАУЗ ТО «ГП №1»,врач-акушер-гинеколог,731.0,1.0
3,ГАУЗ ТО «ГП №1»,врач-оториноларинголог,223.0,1.0
4,ГАУЗ ТО «ГП №1»,врач-офтальмолог,141.0,1.0
...,...,...,...,...
231,ГБУЗ ТО «ОБ №20»,врач-офтальмолог,100.0,1.0
232,ГБУЗ ТО «ОБ №20»,врач-стоматолог,48.0,1.0
233,ГБУЗ ТО «ОБ №20»,врач-стоматолог-терапевт,40.0,1.0
234,ГБУЗ ТО «ОБ №20»,врач-хирург,64.0,1.0


In [107]:
indicator_mo_uchet[indicator_mo_uchet['Организация'] == 'ГБУЗ ТО «ОБ №4»']

,Организация,Должность,Общее число слотов в расписании,Проверка
170,ГБУЗ ТО «ОБ №4»,врач-терапевт участковый,4332.0,1.0
171,ГБУЗ ТО «ОБ №4»,врач-педиатр участковый,2434.0,1.0
172,ГБУЗ ТО «ОБ №4»,врач-акушер-гинеколог,758.0,1.0
173,ГБУЗ ТО «ОБ №4»,врач - детский хирург,48.0,1.0
174,ГБУЗ ТО «ОБ №4»,врач-оториноларинголог,454.0,1.0
175,ГБУЗ ТО «ОБ №4»,врач-офтальмолог,618.0,1.0
176,ГБУЗ ТО «ОБ №4»,врач-психиатр-нарколог,15.0,1.0
177,ГБУЗ ТО «ОБ №4»,врач-стоматолог,741.0,1.0
178,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,71.0,1.0
179,ГБУЗ ТО «ОБ №4»,врач-стоматолог-терапевт,382.0,1.0


1. Расчет показателя 'Доля фактического количества слотов от расчетных, %'

In [108]:
fact_slots = pd.pivot_table(indicator_mo_uchet, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

In [109]:
itog_slots = pd.merge(fact_slots, pivot_table_frmr, on='Организация')

In [110]:
itog_slots['Доля фактического количества слотов от расчетных, не менее 80%'] = round(itog_slots['Общее число слотов в расписании'] / itog_slots['Сумма слотов на врача'] * 100, 2)
itog_slots = itog_slots[['Организация', 'Доля фактического количества слотов от расчетных, не менее 80%']]
itog_slots = itog_slots[itog_slots['Организация'] != 'ПЦ_Тюмень']
itog_slots = itog_slots[itog_slots['Организация'] != 'ОКБ1']
itog_slots.reset_index(drop=True, inplace=True)
itog_slots

,Организация,"Доля фактического количества слотов от расчетных, не менее 80%"
0,"ГАУЗ ТО ""ОСП""",87.23
1,ГАУЗ ТО «ГП №12»,61.63
2,ГАУЗ ТО «ГП №13»,81.46
3,ГАУЗ ТО «ГП №17»,68.05
4,ГАУЗ ТО «ГП №1»,67.47
5,ГАУЗ ТО «ГП №3»,71.46
6,ГАУЗ ТО «ГП №4»,58.96
7,ГАУЗ ТО «ГП №5»,72.65
8,ГАУЗ ТО «ГП №6»,75.61
9,ГАУЗ ТО «ГП №8»,77.04


2. Расчет показателя 'Доля фактического количества слотов от расчетных у врачей участковой службы, %'

##### считаем норму для участковой службы

In [111]:
frmr_uchastkov = frmr_all_merge[(frmr_all_merge['Должность'] ==  'врач-педиатр участковый') |  (frmr_all_merge['Должность'] ==  'врач-терапевт участковый')]

In [112]:
frmr_uchastkov = pd.merge(frmr_uchastkov, norma, how='left', on='Должность')

In [113]:
frmr_uchastkov['Норма штатных единиц'] = frmr_uchastkov['Норма слотов'] * frmr_uchastkov['Занятых штатных единиц']

In [114]:
frmr_uchastkov

,Организация,Структурное подразделение,Отделение стационара,Должность,Штатных единиц,Занятых штатных единиц,В т.ч. внешнее совместительство,Вакансий,Норма слотов,Норма штатных единиц
0,ГАУЗ ТО «ГП №1»,Терапевтическое отделение,0,врач-терапевт участковый,21.0,21.0,0.0,0.0,146,3066.0
1,ГАУЗ ТО «ГП №1»,Педиатрическое отделение,0,врач-педиатр участковый,22.0,22.0,0.0,0.0,145,3190.0
2,ГАУЗ ТО «ГП №12»,Терапевтическое отделение №1,0,врач-терапевт участковый,9.0,9.0,0.0,0.0,146,1314.0
3,ГАУЗ ТО «ГП №12»,Терапевтическое отделение №2,0,врач-терапевт участковый,9.0,9.0,0.0,0.0,146,1314.0
4,ГАУЗ ТО «ГП №12»,Лечебно-диагностическое отделение № 7,0,врач-терапевт участковый,2.0,2.0,0.0,0.0,146,292.0
...,...,...,...,...,...,...,...,...,...,...
175,NaN,Поликлиника №2,0,врач-терапевт участковый,2.0,2.0,0.0,0.0,146,292.0
176,NaN,Поликлиника №1,0,врач-педиатр участковый,2.0,2.0,0.0,0.0,145,290.0
177,NaN,Поликлиника №1,0,врач-терапевт участковый,2.0,1.0,0.0,1.0,146,146.0
178,NaN,Детская поликлиника,0,врач-педиатр участковый,6.0,5.0,0.0,1.0,145,725.0


In [115]:
pivot_table_frmr_uchastkov = pd.pivot_table(frmr_uchastkov, index='Организация', values='Норма штатных единиц', aggfunc=np.sum, margins=True, margins_name='Итого')
pivot_table_frmr_uchastkov = pivot_table_frmr_uchastkov.reset_index()

In [116]:
pivot_table_frmr_uchastkov = pivot_table_frmr_uchastkov[['Организация', 'Норма штатных единиц']]

###### считаем факт для участковой службы

In [117]:
indicator_uchaskov = indicator_mo[(indicator_mo['Должность'] ==  'врач-педиатр участковый') |  (indicator_mo['Должность'] ==  'врач-терапевт участковый')]

In [118]:
indicator_mo_uchet_uchastkov = pd.merge(indicator_uchaskov, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [119]:
indicator_mo_uchet_uchastkov = indicator_mo_uchet_uchastkov[indicator_mo_uchet_uchastkov['Проверка'] != 0]

In [120]:
fact_slots_uchaskov = pd.pivot_table(indicator_mo_uchet_uchastkov, index='Организация', values='Общее число слотов в расписании', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

In [121]:
itog_slots_uchastkov = pd.merge(fact_slots_uchaskov, pivot_table_frmr_uchastkov, on='Организация')
itog_slots_uchastkov['Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%'] = round(itog_slots_uchastkov['Общее число слотов в расписании'] / itog_slots_uchastkov['Норма штатных единиц'] * 100, 2)
itog_slots_uchastkov = itog_slots_uchastkov[['Организация', 'Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%']]
itog_slots_uchastkov.tail(1)

,Организация,"Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%"
22,Итого,64.99


###### Слайд  2.2.Считаем долю конкурентных слотов от общего количества слотов, %

In [122]:
# Загружаю расписание 1С
# guide_mo_1c = pd.read_excel(GUIDE_MEDORGANIZATION)
ras_1c = pd.read_excel(RASPISANIE_1C, usecols=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type', 'date_report',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'])

In [123]:
# Загружаю поправочную таблицу 
delRas1c = pd.read_excel(DELETE_RAS_1C, usecols=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'])

In [124]:
# Объединяю таблицы
ras1c_merged = pd.merge(ras_1c, delRas1c, on=['region', 'mo_oid', 'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 
                                              'mp_fio', 'mp_dolgnost', 'mp_stavka', 'slots_type',
                                              'slots_referral', 'slot_lenght', 'slots_created', 'slots_booked',
                                              'slots_free_after_booked', 'visits_absence', 'visits_success'], how='left', indicator=True)

In [125]:
ras1c_filtered = ras1c_merged[ras1c_merged['_merge'] == 'left_only'].drop(columns='_merge')

In [126]:
ras1c_filtered 

,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,date_report,mp_dolgnost,mp_stavka,slots_type,slots_referral,slot_lenght,slots_created,slots_booked,slots_free_after_booked,visits_absence,visits_success
0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,Не доступен,нет,10,4,4,0,0,4
1,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,Не доступен,для повторной записи,12,2,2,0,0,2
2,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,Не доступен,нет,20,4,4,0,0,4
3,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,Доступен,нет,20,13,13,0,0,13
4,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-13,врач-акушер-гинеколог,1.0,Не доступен,нет,10,3,3,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87586,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-07,врач-стоматолог-терапевт,1.0,Доступен,нет,30,6,6,0,0,0
87587,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-07,врач-стоматолог-терапевт,1.0,Не доступен,для повторной записи,30,4,0,0,0,0
87588,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-09,врач-стоматолог-терапевт,1.0,Не доступен,для повторной записи,30,4,0,0,0,0
87589,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-09,врач-стоматолог-терапевт,1.0,Доступен,нет,30,6,6,0,0,0


In [127]:
primer  = ras1c_filtered['mo_short_name'].unique()
primer_mo_name = []
for mo_name in primer:
    primer_mo_name.append(mo_name)

In [128]:
df_primer_mo_name  = pd.DataFrame({'mo_short_name': primer_mo_name})

In [129]:
merge_ras_1c = pd.merge(ras1c_filtered, medorganization, on='mo_short_name', how='left')

In [130]:
merge_ras_1c.rename(columns={'mp_dolgnost': 'Должность'}, inplace=True)

In [131]:
merge_ras_1c

,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,date_report,Должность,mp_stavka,...,slots_booked,slots_free_after_booked,visits_absence,visits_success,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,...,4,0,0,4,"ГАУЗ ТО ""Областная больница №19""",ГБУЗ ТО «ОБ №19»,ОБ19,"ГАУЗ ТО ""Областная больница №19""","ГАУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №19""",ГАУЗ ТО «Областная больница № 19»
1,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,...,2,0,0,2,"ГАУЗ ТО ""Областная больница №19""",ГБУЗ ТО «ОБ №19»,ОБ19,"ГАУЗ ТО ""Областная больница №19""","ГАУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №19""",ГАУЗ ТО «Областная больница № 19»
2,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,...,4,0,0,4,"ГАУЗ ТО ""Областная больница №19""",ГБУЗ ТО «ОБ №19»,ОБ19,"ГАУЗ ТО ""Областная больница №19""","ГАУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №19""",ГАУЗ ТО «Областная больница № 19»
3,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-12,врач-акушер-гинеколог,1.0,...,13,0,0,13,"ГАУЗ ТО ""Областная больница №19""",ГБУЗ ТО «ОБ №19»,ОБ19,"ГАУЗ ТО ""Областная больница №19""","ГАУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №19""",ГАУЗ ТО «Областная больница № 19»
4,Тюменская область,1.2.643.5.1.13.13.12.2.72.7305,ГАУЗ ТО «Областная больница № 19»,1.2.643.5.1.13.13.12.2.72.7305.0.47750,Богандинская поликлиника,10475381849,Абдухалимова Нонна Анатольевна,2024-02-13,врач-акушер-гинеколог,1.0,...,3,0,0,3,"ГАУЗ ТО ""Областная больница №19""",ГБУЗ ТО «ОБ №19»,ОБ19,"ГАУЗ ТО ""Областная больница №19""","ГАУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №19""",ГАУЗ ТО «Областная больница № 19»
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87570,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-07,врач-стоматолог-терапевт,1.0,...,6,0,0,0,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ОСП,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ГАУЗ ТО «Областная стоматологическая поликлиника»
87571,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-07,врач-стоматолог-терапевт,1.0,...,0,0,0,0,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ОСП,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ГАУЗ ТО «Областная стоматологическая поликлиника»
87572,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-09,врач-стоматолог-терапевт,1.0,...,0,0,0,0,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ОСП,"ГАУЗ ТО ""Областная стоматологическая поликлиника""","ГАУЗ ТО ""ОСП""",ГАУЗ ТО «Областная стоматологическая поликлиника»
87573,Тюменская область,1.2.643.5.1.13.13.12.2.72.7357,ГАУЗ ТО «Областная стоматологическая поликлиника»,1.2.643.5.1.13.13.12.2.72.7357.0.423403,"ЛПО-3, Ленина 9",15759057404,Эюбова Айнуре Али-Кызы,2024-03-09,врач-стоматолог-терапевт,1.0,...,6,0,0,0,"ГАУЗ ТО ""Об

In [132]:
merge_ras_1c_uchet = pd.merge(employee_uchet, merge_ras_1c, on=['Организация', 'Должность']).fillna(0)

In [133]:
merge_ras_1c_uchet = merge_ras_1c_uchet[merge_ras_1c_uchet['Проверка'] != 0]

In [134]:
merge_ras_1c_uchet.reset_index(drop=True, inplace=True)

In [135]:
# Отбираем строки в DataFrame, попадающие в заданный период
merge_ras_1c_uchet = merge_ras_1c_uchet[(merge_ras_1c_uchet['date_report'] >= start_date) & (merge_ras_1c_uchet['date_report'] <= end_date)]

In [136]:
merge_ras_1c_uchet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21577 entries, 10 to 87459
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Организация                       21577 non-null  object        
 1   Должность                         21577 non-null  object        
 2   Проверка                          21577 non-null  float64       
 3   region                            21577 non-null  object        
 4   mo_oid                            21577 non-null  object        
 5   mo_short_name                     21577 non-null  object        
 6   sp_oid                            21577 non-null  object        
 7   sp_name                           21577 non-null  object        
 8   mp_id                             21577 non-null  int64         
 9   mp_fio                            21577 non-null  object        
 10  date_report                       21577 non-n

In [137]:
merge_ras_1c_uchet['date_report'].unique()

array(['2024-02-19T00:00:00.000000000', '2024-02-20T00:00:00.000000000',
       '2024-02-21T00:00:00.000000000', '2024-02-22T00:00:00.000000000',
       '2024-02-24T00:00:00.000000000', '2024-02-23T00:00:00.000000000',
       '2024-02-25T00:00:00.000000000'], dtype='datetime64[ns]')

##### Расписание из 1С выгружено и обработано

In [138]:
merge_ras_1c_uchet

,Организация,Должность,Проверка,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,...,slots_created,slots_booked,slots_free_after_booked,visits_absence,visits_success,Наименование МО,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
10,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
11,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,6,6,0,0,6,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
12,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
13,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,7,7,0,0,7,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
14,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,4,4,0,0,4,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87455,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,12,12,0,0,12,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87456,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87457,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,11,11,0,0,11,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87458,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""


In [139]:
merge_ras_1c_uchet.columns

Index(['Организация', 'Должность', 'Проверка', 'region', 'mo_oid',
       'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 'mp_fio', 'date_report',
       'mp_stavka', 'slots_type', 'slots_referral', 'slot_lenght',
       'slots_created', 'slots_booked', 'slots_free_after_booked',
       'visits_absence', 'visits_success', 'Наименование МО', 'МО',
       'МедОрганизация', 'Краткое наименование организации',
       'Медицинская организация'],
      dtype='object')

In [140]:
pivot_ras_1c = pd.pivot_table(merge_ras_1c_uchet, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

In [141]:
pivot_ras_1c['Доля конкурентных слотов от общего количества слотов, не менее 65%'] = round(pivot_ras_1c['Доступен'] / (pivot_ras_1c['Доступен'] + pivot_ras_1c['Не доступен']) * 100, 2)
pivot_ras_1c_s2 = pivot_ras_1c[['Организация', 'Доля конкурентных слотов от общего количества слотов, не менее 65%']]
pivot_ras_1c_s2 = pivot_ras_1c_s2.rename_axis(None, axis=1)
pivot_ras_1c_s2

,Организация,"Доля конкурентных слотов от общего количества слотов, не менее 65%"
0,"ГАУЗ ТО ""ОСП""",51.85
1,ГАУЗ ТО «ГП №12»,48.91
2,ГАУЗ ТО «ГП №13»,57.84
3,ГАУЗ ТО «ГП №17»,61.13
4,ГАУЗ ТО «ГП №1»,58.16
5,ГАУЗ ТО «ГП №3»,53.06
6,ГАУЗ ТО «ГП №4»,55.43
7,ГАУЗ ТО «ГП №5»,62.14
8,ГАУЗ ТО «ГП №6»,64.99
9,ГАУЗ ТО «ГП №8»,58.38


4. Доля конкурентных слотов участковой службы от общего количества слотов

In [142]:
ras_1c_uchaskov = merge_ras_1c_uchet[(merge_ras_1c_uchet['Должность'] ==  'врач-педиатр участковый') |  (merge_ras_1c_uchet['Должность'] ==  'врач-терапевт участковый')]
pivot_ras_1c_uchstkov = pd.pivot_table(ras_1c_uchaskov, index='Организация', columns = 'slots_type', values= 'slots_created', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
pivot_ras_1c_uchstkov['Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%'] = round(pivot_ras_1c_uchstkov['Доступен'] / (pivot_ras_1c_uchstkov['Доступен'] + pivot_ras_1c_uchstkov['Не доступен']) * 100, 2)
pivot_ras_1c_uchstkov.reset_index(drop=True, inplace= True)

In [143]:
pivot_ras_1c_uchstkov_s2 = pivot_ras_1c_uchstkov[['Организация', 'Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%']]

In [144]:
pivot_ras_1c_uchstkov_s2 = pivot_ras_1c_uchstkov_s2.rename_axis(None, axis=1)
pivot_ras_1c_uchstkov_s2.head(1)

,Организация,"Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №12»,53.68


* Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%

In [145]:
input_analit_filtr = pd.read_excel(FILTR_ANALIT)

In [146]:
input_analit_filtr = input_analit_filtr[(input_analit_filtr['Медицинская организация'] != 'Всего')]


In [147]:
analit_merge_filtr = pd.merge(input_analit_filtr, medorganization, on='Медицинская организация', how='left')

In [148]:
analit_merge_filtr.columns

Index(['Медицинская организация', 'Отделение', 'Отделение (адрес)',
       'Должность', 'Специальность', 'Врач', 'Дней расписания',
       'Слотов на 7 дней', 'Слотов на 14 дней', 'Слотов на 30 дней',
       'Формиров. расписания на 7 дней, %',
       'Формиров. расписания на 14 дней, %',
       'Формиров. расписания на 30 дней, %',
       'Слотов первичного приёма на 7 дней',
       'Доля первичного приёма на 7 дней, %',
       'Слотов первичного приёма на 14 дней',
       'Доля первичного приёма на 14 дней, %',
       'Слотов первичного приёма на 30 дней',
       'Доля первичного приёма на 30 дней, %', 'Наименование МО',
       'Организация', 'МО', 'МедОрганизация',
       'Краткое наименование организации', 'mo_short_name'],
      dtype='object')

In [149]:
delete_rows3 = pd.read_excel(GUIDE_DELETE_MED, skiprows=6)
delete_rows3.rename(columns={'Unnamed: 0': 'СНИЛС', 
                    'Unnamed: 1': 'Фамилия', 
                    'Unnamed: 2': 'Имя', 
                    'Unnamed: 3': 'Отчество', 
                    'Unnamed: 4': 'Пол', 
                    'Unnamed: 5': 'Дата рождения', 
                    'Unnamed: 6': 'Сведения о трудоустройстве'}, inplace=True)

In [150]:
delete_rows3 = delete_rows3[['Краткое наименование организации', 'Наименование структурного подразделения', 'Должность по фед справочнику']]
delete_rows3 = delete_rows3.rename(columns={'Должность по фед справочнику' : 'Должность', 'Наименование структурного подразделения': 'Отделение'})
delete_rows3['Удалить'] = 'True'

In [151]:
delete_rows3.head(1)

,Краткое наименование организации,Отделение,Должность,Удалить
0,"ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №3""",Отделение организации медицинской помощи студе...,врач-офтальмолог,True


In [152]:
result_original_filtr = pd.merge(analit_merge_filtr, delete_rows3, on=['Краткое наименование организации', 
                                      'Отделение', 
                                      'Должность'], how='left')
result_original_filtr = result_original_filtr[result_original_filtr['Удалить'] != 'True']

##### Подгружаем таблицу должностей и мержим ее с таблицей из Аналит

##### Это таблица должностей аналит

In [153]:
table_analit_doljnost

,Должность,Специальность
0,врач-терапевт участковый,Лечебное дело
1,врач-терапевт участковый,Терапия
2,врач-педиатр участковый,Педиатрия
3,врач общей практики (семейный врач),Общая врачебная практика (семейная медицина)
4,врач общей практики (семейный врач),Лечебное дело
5,врач-офтальмолог,Офтальмология
6,врач-оториноларинголог,Оториноларингология
7,врач-хирург,Хирургия
8,врач-акушер-гинеколог,Акушерство и гинекология
9,врач-психиатр-нарколог,Психиатрия-наркология


In [154]:
merge_table_analit_result_filter = pd.merge(result_original_filtr, table_analit_doljnost, on = ['Должность', 'Специальность'], how='right')

In [155]:
merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'] = merge_table_analit_result_filter['Доля первичного приёма на 7 дней, %'].replace('-', 0)

In [156]:
merge_table_analit_result_filter

,Медицинская организация,Отделение,Отделение (адрес),Должность,Специальность,Врач,Дней расписания,Слотов на 7 дней,Слотов на 14 дней,Слотов на 30 дней,...,"Доля первичного приёма на 14 дней, %",Слотов первичного приёма на 30 дней,"Доля первичного приёма на 30 дней, %",Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,mo_short_name,Удалить
0,ГАУЗ ТО «ГП № 1»,Терапевтическое отделение,"625022, Тюменская обл, Тюмень г, Газовиков ул,...",врач-терапевт участковый,Лечебное дело,Азизов Руслан Азизович,10.0,136.0,242.0,270.0,...,0.702479,190.0,0.703704,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,NaN
1,ГАУЗ ТО «ГП № 1»,Терапевтическое отделение,"625022, Тюменская обл, Тюмень г, Газовиков ул,...",врач-терапевт участковый,Лечебное дело,Алгинина Валентина Дмитриевна,10.0,133.0,236.0,265.0,...,0.872881,232.0,0.875472,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,NaN
2,ГАУЗ ТО «ГП № 1»,Терапевтическое отделение,"625022, Тюменская обл, Тюмень г, Газовиков ул,...",врач-терапевт участковый,Лечебное дело,Ватаманюк Галина Викторовна,10.0,136.0,242.0,270.0,...,0.702479,190.0,0.703704,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,NaN
3,ГАУЗ ТО «ГП № 1»,Терапевтическое отделение,"625022, Тюменская обл, Тюмень г, Газовиков ул,...",врач-терапевт участковый,Лечебное дело,Гетман Гульнара Маратовна,10.0,136.0,242.0,270.0,...,0.702479,190.0,0.703704,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,NaN
4,ГАУЗ ТО «ГП № 1»,Терапевтическое отделение,"625022, Тюменская обл, Тюмень г, Газовиков ул,...",врач-терапевт участковый,Лечебное дело,Ерохина Надежда Викторовна,5.0,0.0,106.0,134.0,...,0.698113,94.0,0.701493,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,ГБУЗ ТО «Областная больница № 24»,Кабинет врача-психиатра,"Тюменская обл, м.р-н Ярковский, с.п. Ярковское...",врач-психиатр,Психиатрия,Макарова Татьяна Викторовна - (Врач-психиатр),7.0,72.0,136.0,160.0,...,0.720588,116.0,0.725000,ГБУЗ ТО ОБ №24 (с.Ярково),ГБУЗ ТО «ОБ №24»,ОБ24,"ГБУЗ ТО ""Областная больница №24"" (с.Ярково)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №24"" (С. ЯРКОВО)",ГБУЗ ТО «Областная больница № 24»,NaN
1824,ГБУЗ ТО «Областная клиническая психиатрическая...,Взрослое психоневрологическое отделение,"625000, Тюменская обл, Тюмень г, Червишевский ...",врач-психиатр,Психиатрия,Косинцева Юлия Витальевна,9.0,88.0,152.0,152.0,...,0,0.0,0.000000,"ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «ОКПБ»,ОКПБ,"ГБУЗ ТО ""Областная клиническая психиатрическая...","ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «Областная клиническая психиатрическая...,NaN
1825,ГБУЗ ТО «Областная клиническая психиатрическая...,Взрослое психоневрологическое отделение,"625000, Тюменская обл, Тюмень г, Червишевский ...",врач-психиатр,Психиатрия,Пешкова Елизавета Алексеевна,9.0,43.0,76.0,76.0,...,0,0.0,0.000000,"ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «ОКПБ»,ОКПБ,"ГБУЗ ТО ""Областная клиническая психиатрическая...","ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «Областная клиническая психиатрическая...,NaN
1826,ГБУЗ ТО «Областная клиническая психиатрическая...,Взрослое психоневрологическое отделение,"625000, Тюменская обл, Тюмень г, Червишевский ...",врач-психиатр,Психиатрия,Темерева Вера Юрьевна,9.0,5.0,9.0,9.0,...,0,0.0,0.000000,"ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «ОКПБ»,ОКПБ,"ГБУЗ ТО ""Областная клиническая психиатрическая...","ГБУЗ ТО ""ОКПБ""",ГБУЗ ТО «Областная клиническая психиатрическая.

In [157]:
merge_table_analit_result_filter = merge_table_analit_result_filter[(merge_table_analit_result_filter['Должность'] == 'врач-терапевт участковый') | 
                          (merge_table_analit_result_filter['Должность'] == 'врач-педиатр участковый')]

In [158]:
svod_slotov = pd.pivot_table(df1, values=['Слотов на 7 дней', 
                                          'Слотов первичного приёма на 7 дней'], 
                             index=['Организация'], 
                             aggfunc=np.sum, 
                             fill_value=0, margins=True, margins_name='Итого')

In [159]:
svod_slotov.reset_index(drop=False, inplace=True)

In [160]:
def proz7days(row):
    if row['Слотов первичного приёма на 7 дней'] != 0 and row['Слотов на 7 дней'] != 0:
        val = round(row['Слотов первичного приёма на 7 дней'] / row['Слотов на 7 дней'] * 100, 2)
    else:
        val = 0
    return val

In [161]:
svod_slotov['% конкурентных слотов на 7 дней'] = svod_slotov.apply(proz7days, axis = 1)

In [162]:
svod_slotov = svod_slotov[['Организация', '% конкурентных слотов на 7 дней']]

In [163]:
svod_slotov.rename(columns={'% конкурентных слотов на 7 дней': 'Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%'}, inplace=True)

##### Итог расчета слайда 2

In [164]:
itog_slots

,Организация,"Доля фактического количества слотов от расчетных, не менее 80%"
0,"ГАУЗ ТО ""ОСП""",87.23
1,ГАУЗ ТО «ГП №12»,61.63
2,ГАУЗ ТО «ГП №13»,81.46
3,ГАУЗ ТО «ГП №17»,68.05
4,ГАУЗ ТО «ГП №1»,67.47
5,ГАУЗ ТО «ГП №3»,71.46
6,ГАУЗ ТО «ГП №4»,58.96
7,ГАУЗ ТО «ГП №5»,72.65
8,ГАУЗ ТО «ГП №6»,75.61
9,ГАУЗ ТО «ГП №8»,77.04


In [165]:
pivot_ras_1c_s2

,Организация,"Доля конкурентных слотов от общего количества слотов, не менее 65%"
0,"ГАУЗ ТО ""ОСП""",51.85
1,ГАУЗ ТО «ГП №12»,48.91
2,ГАУЗ ТО «ГП №13»,57.84
3,ГАУЗ ТО «ГП №17»,61.13
4,ГАУЗ ТО «ГП №1»,58.16
5,ГАУЗ ТО «ГП №3»,53.06
6,ГАУЗ ТО «ГП №4»,55.43
7,ГАУЗ ТО «ГП №5»,62.14
8,ГАУЗ ТО «ГП №6»,64.99
9,ГАУЗ ТО «ГП №8»,58.38


In [166]:
itog_slots_uchastkov

,Организация,"Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%"
0,ГАУЗ ТО «ГП №12»,61.28
1,ГАУЗ ТО «ГП №13»,80.96
2,ГАУЗ ТО «ГП №17»,67.25
3,ГАУЗ ТО «ГП №1»,70.59
4,ГАУЗ ТО «ГП №3»,68.35
5,ГАУЗ ТО «ГП №4»,61.02
6,ГАУЗ ТО «ГП №5»,76.29
7,ГАУЗ ТО «ГП №6»,76.87
8,ГАУЗ ТО «ГП №8»,72.61
9,ГБУЗ ТО «ОБ №11»,61.47


In [167]:
svod_slotov

,Организация,"Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%"
0,"ГАУЗ ТО ""ОСП""",64.30
1,ГАУЗ ТО «ГП №12»,72.11
2,ГАУЗ ТО «ГП №13»,66.04
3,ГАУЗ ТО «ГП №17»,75.73
4,ГАУЗ ТО «ГП №1»,65.54
5,ГАУЗ ТО «ГП №3»,67.88
6,ГАУЗ ТО «ГП №4»,65.37
7,ГАУЗ ТО «ГП №5»,67.88
8,ГАУЗ ТО «ГП №6»,70.74
9,ГАУЗ ТО «ГП №8»,75.60


In [168]:
# Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%
pivot_ras_1c_uchstkov_s2

,Организация,"Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №12»,53.68
1,ГАУЗ ТО «ГП №13»,59.97
2,ГАУЗ ТО «ГП №17»,64.37
3,ГАУЗ ТО «ГП №1»,58.35
4,ГАУЗ ТО «ГП №3»,53.08
5,ГАУЗ ТО «ГП №4»,52.33
6,ГАУЗ ТО «ГП №5»,64.38
7,ГАУЗ ТО «ГП №6»,63.60
8,ГАУЗ ТО «ГП №8»,60.74
9,ГБУЗ ТО «ОБ №11»,56.86


In [169]:
data_frames = [itog_slots, pivot_ras_1c_s2, itog_slots_uchastkov, svod_slotov, pivot_ras_1c_uchstkov_s2]

In [170]:
pivot_ras_1c_s2 = reduce(lambda  left,right: pd.merge(left,right,on=['Организация'], how='outer'), data_frames)

In [171]:
pivot_ras_1c_s2

,Организация,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,"ГАУЗ ТО ""ОСП""",87.23,51.85,NaN,64.30,NaN
1,ГАУЗ ТО «ГП №12»,61.63,48.91,61.28,72.11,53.68
2,ГАУЗ ТО «ГП №13»,81.46,57.84,80.96,66.04,59.97
3,ГАУЗ ТО «ГП №17»,68.05,61.13,67.25,75.73,64.37
4,ГАУЗ ТО «ГП №1»,67.47,58.16,70.59,65.54,58.35
5,ГАУЗ ТО «ГП №3»,71.46,53.06,68.35,67.88,53.08
6,ГАУЗ ТО «ГП №4»,58.96,55.43,61.02,65.37,52.33
7,ГАУЗ ТО «ГП №5»,72.65,62.14,76.29,67.88,64.38
8,ГАУЗ ТО «ГП №6»,75.61,64.99,76.87,70.74,63.60
9,ГАУЗ ТО «ГП №8»,77.04,58.38,72.61,75.60,60.74


In [172]:
result_slide2 = pivot_ras_1c_s2[['Организация', 'Доля фактического количества слотов от расчетных, не менее 80%', 
                                 'Доля конкурентных слотов от общего количества слотов, не менее 65%', 
                                 'Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%', 
                                 'Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%',
                                 'Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%']]
result_slide2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide2.fillna(0, inplace=True)

In [173]:
result_slide2

,Наименование МО,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,"ГАУЗ ТО ""ОСП""",87.23,51.85,0.00,64.30,0.00
1,ГАУЗ ТО «ГП №12»,61.63,48.91,61.28,72.11,53.68
2,ГАУЗ ТО «ГП №13»,81.46,57.84,80.96,66.04,59.97
3,ГАУЗ ТО «ГП №17»,68.05,61.13,67.25,75.73,64.37
4,ГАУЗ ТО «ГП №1»,67.47,58.16,70.59,65.54,58.35
5,ГАУЗ ТО «ГП №3»,71.46,53.06,68.35,67.88,53.08
6,ГАУЗ ТО «ГП №4»,58.96,55.43,61.02,65.37,52.33
7,ГАУЗ ТО «ГП №5»,72.65,62.14,76.29,67.88,64.38
8,ГАУЗ ТО «ГП №6»,75.61,64.99,76.87,70.74,63.60
9,ГАУЗ ТО «ГП №8»,77.04,58.38,72.61,75.60,60.74


In [174]:
# Определяем значение, по которому будем перемещать строку в конец
value_to_move = 'Итого'

# Выбираем строку по заданному значению
row_to_move = result_slide2[result_slide2['Наименование МО'] == value_to_move]

In [175]:
row_to_move

,Наименование МО,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
28,Итого,67.66,57.46,64.99,69.07,58.53


In [176]:
# Удаляем эту строку из исходного места
result_slide2 = result_slide2.drop(row_to_move.index)

In [177]:
result_slide2.reset_index(drop=True, inplace=True)

In [178]:
sequence_mo2 = sequence_mo.rename(columns={'Организация': 'Наименование МО'})

In [179]:
sequence_mo2 = sequence_mo2[sequence_mo2['Наименование МО'] != 'Итог']

In [180]:
sequence_mo2

,Наименование МО
0,ГАУЗ ТО «ГП №1»
1,ГАУЗ ТО «ГП №12»
2,ГАУЗ ТО «ГП №13»
3,ГАУЗ ТО «ГП №17»
4,ГАУЗ ТО «ГП №3»
5,ГАУЗ ТО «ГП №4»
6,ГАУЗ ТО «ГП №5»
7,ГАУЗ ТО «ГП №6»
8,ГАУЗ ТО «ГП №8»
9,"ГАУЗ ТО ""ОСП"""


In [181]:
df2 = pd.merge(sequence_mo2, result_slide2, how='left', on='Наименование МО')

In [182]:
df2

,Наименование МО,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №1»,67.47,58.16,70.59,65.54,58.35
1,ГАУЗ ТО «ГП №12»,61.63,48.91,61.28,72.11,53.68
2,ГАУЗ ТО «ГП №13»,81.46,57.84,80.96,66.04,59.97
3,ГАУЗ ТО «ГП №17»,68.05,61.13,67.25,75.73,64.37
4,ГАУЗ ТО «ГП №3»,71.46,53.06,68.35,67.88,53.08
5,ГАУЗ ТО «ГП №4»,58.96,55.43,61.02,65.37,52.33
6,ГАУЗ ТО «ГП №5»,72.65,62.14,76.29,67.88,64.38
7,ГАУЗ ТО «ГП №6»,75.61,64.99,76.87,70.74,63.60
8,ГАУЗ ТО «ГП №8»,77.04,58.38,72.61,75.60,60.74
9,"ГАУЗ ТО ""ОСП""",87.23,51.85,0.00,64.30,0.00


In [183]:
# Добавляем эту строку в конец DataFrame
df2 = pd.concat([df2, row_to_move])


In [184]:
df2.reset_index(drop=True, inplace=True)

In [185]:
df2

,Наименование МО,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №1»,67.47,58.16,70.59,65.54,58.35
1,ГАУЗ ТО «ГП №12»,61.63,48.91,61.28,72.11,53.68
2,ГАУЗ ТО «ГП №13»,81.46,57.84,80.96,66.04,59.97
3,ГАУЗ ТО «ГП №17»,68.05,61.13,67.25,75.73,64.37
4,ГАУЗ ТО «ГП №3»,71.46,53.06,68.35,67.88,53.08
5,ГАУЗ ТО «ГП №4»,58.96,55.43,61.02,65.37,52.33
6,ГАУЗ ТО «ГП №5»,72.65,62.14,76.29,67.88,64.38
7,ГАУЗ ТО «ГП №6»,75.61,64.99,76.87,70.74,63.60
8,ГАУЗ ТО «ГП №8»,77.04,58.38,72.61,75.60,60.74
9,"ГАУЗ ТО ""ОСП""",87.23,51.85,0.00,64.30,0.00


##### Подготовка данных для слайда 3

1. Расчет первого столбца 'Занятость слотов всего (доля от общего числа созданных и количества занятых)'
* Использую уже сверенный с проверочными файлами результат обработки расписания из 1С

In [186]:
slide_3_column_1 = merge_ras_1c_uchet
slide_3_column_1.rename(columns={'Должность': 'mp_dolgnost'}, inplace=True)

In [187]:
merge_ras_1c_uchet['date_report'].unique()

array(['2024-02-19T00:00:00.000000000', '2024-02-20T00:00:00.000000000',
       '2024-02-21T00:00:00.000000000', '2024-02-22T00:00:00.000000000',
       '2024-02-24T00:00:00.000000000', '2024-02-23T00:00:00.000000000',
       '2024-02-25T00:00:00.000000000'], dtype='datetime64[ns]')

In [188]:
slide_3_column_1 = slide_3_column_1[['МО', 'mp_dolgnost', 'slots_type', 'slots_created', 'slots_booked']]

In [189]:
slide_3_column_1_pivot = pd.pivot_table(slide_3_column_1, index='МО', 
                                        columns='slots_type', 
                                        values=['slots_created', 'slots_booked'],
                                        aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

In [190]:
slide_3_column_1_pivot

МО slots_booked                     slots_created              \
slots_type            Доступен Не доступен   Итого      Доступен Не доступен   
0             ГП1         3490        2153    5643          4045        2910   
1            ГП12         7938        9163   17101          9353        9770   
2            ГП13         2741        1885    4626          3274        2386   
3            ГП17         7560        4759   12319          8236        5238   
4             ГП3         5435        4889   10324          6352        5619   
5             ГП4         3436        2759    6195          4012        3226   
6             ГП5        12324        6723   19047         13821        8419   
7             ГП6         3604        1561    5165          4392        2366   
8             ГП8         4603        4666    9269          7301        5206   
9            ОБ11         3261        2875    6136          4521        3856   
10           ОБ12         2570        2604    5174          4818        3223   
11           ОБ13          588         620    1208           853         775   
12           ОБ14         1041         787    1828          1508        1093   
13           ОБ15         1218         702    1920          1602         904   
14           ОБ19         6014        5058   11072          7363        6769   
15           ОБ20          966         481    1447          1069         556   
16           ОБ23         3589        1376    4965          4316        1712   
17           ОБ24          834        1228    2062          1413        1593   
18            ОБ3         4660        3228    7888          5204        3504   
19            ОБ4         5882        3102    8984          8247        4312   
20            ОБ9          761         535    1296          1226         742   
21           ОКБ2         2388        3385    5773          2762        4633   
22           ОКПБ          221         195     416           222         195   
23           ОКФЦ         1043         497    1540          1148         589   
24            ОНД          253         105     358           462         236   
25            ОСП         1157        1269    2426          1513        1405   
26            РД2          671         296     967           811         326   
27            РД3          633         263     896           715         301   
28          Итого        88881       67164  156045        110559       81864   

                    
slots_type   Итого  
0             6955  
1            19123  
2             5660  
3            13474  
4            11971  
5             7238  
6            22240  
7             6758  
8            12507  
9             8377  
10            8041  
11            1628  
12            2601  
13            2506  
14           14132  
15            1625  
16            6028  
17            3006  
18            8708  
19           12559  
20            1968  
21            7395  
22             417  
23            1737  
24             698  
25            2918  
26            1137  
27            1016  
28          192423

In [191]:
slide_3_column_1_pivot = slide_3_column_1_pivot.droplevel(0, axis=1)

In [192]:
#В этом примере мы создаем DataFrame с неуникальными именами столбцов. Мы используем метод MultiIndex.from_tuples для создания мультииндексов, чтобы сделать имена столбцов уникальными путем добавления индекса к дублирующимся именам. После этого мы присваиваем новые имена столбцов DataFrame.
slide_3_column_1_pivot.columns = pd.MultiIndex.from_tuples([(col, 'slots_type' + str(i)) if col else (col, '') for i, col in enumerate(slide_3_column_1_pivot.columns)])

In [193]:
slide_3_column_1_pivot = slide_3_column_1_pivot.droplevel(0, axis=1)

In [194]:
slide_3_column_1_pivot = slide_3_column_1_pivot.rename(columns={'': 'МО'})
slide_3_column_1_pivot = slide_3_column_1_pivot[['МО', 'slots_type3', 'slots_type6']]
slide_3_column_1_pivot = slide_3_column_1_pivot.rename_axis(None, axis=1)
slide_3_column_1_pivot['Занятость слотов всего (доля от общего числа созданных и количества занятых)'] = round(slide_3_column_1_pivot['slots_type3'] / slide_3_column_1_pivot['slots_type6'] * 100, 2)

In [195]:
slide_3_column_1_pivot

,МО,slots_type3,slots_type6,Занятость слотов всего (доля от общего числа созданных и количества занятых)
0,ГП1,5643,6955,81.14
1,ГП12,17101,19123,89.43
2,ГП13,4626,5660,81.73
3,ГП17,12319,13474,91.43
4,ГП3,10324,11971,86.24
5,ГП4,6195,7238,85.59
6,ГП5,19047,22240,85.64
7,ГП6,5165,6758,76.43
8,ГП8,9269,12507,74.11
9,ОБ11,6136,8377,73.25


In [196]:
slide_3_column_1_itog = slide_3_column_1_pivot[['МО', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)']]

In [197]:
slide_3_column_1_itog

,МО,Занятость слотов всего (доля от общего числа созданных и количества занятых)
0,ГП1,81.14
1,ГП12,89.43
2,ГП13,81.73
3,ГП17,91.43
4,ГП3,86.24
5,ГП4,85.59
6,ГП5,85.64
7,ГП6,76.43
8,ГП8,74.11
9,ОБ11,73.25


2. 'Доля неявок от общего числа записей'

In [198]:
slide_3_column2 = merge_ras_1c_uchet
slide_3_column2.rename(columns={'Должность': 'mp_dolgnost'}, inplace=True)

In [199]:
slide3_column2 = slide_3_column2[['МО', 'mp_dolgnost', 'slots_type', 'slots_booked', 'visits_absence']]

In [200]:
slide3_column2_pivot = pd.pivot_table(slide3_column2, index='МО', 
                                        columns='slots_type', 
                                        values=['slots_booked', 'visits_absence'],
                                        aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()

In [201]:
slide3_column2_pivot = slide3_column2_pivot.droplevel(0, axis=1)

In [202]:
slide3_column2_pivot

slots_type,,Доступен,Не доступен,Итого,Доступен,Не доступен,Итого
0,ГП1,3490,2153,5643,175,148,323
1,ГП12,7938,9163,17101,511,678,1189
2,ГП13,2741,1885,4626,225,145,370
3,ГП17,7560,4759,12319,1333,966,2299
4,ГП3,5435,4889,10324,448,496,944
5,ГП4,3436,2759,6195,400,247,647
6,ГП5,12324,6723,19047,1758,1196,2954
7,ГП6,3604,1561,5165,346,123,469
8,ГП8,4603,4666,9269,833,603,1436
9,ОБ11,3261,2875,6136,167,342,509


In [203]:
#В этом примере мы создаем DataFrame с неуникальными именами столбцов. Мы используем метод MultiIndex.from_tuples для создания мультииндексов, чтобы сделать имена столбцов уникальными путем добавления индекса к дублирующимся именам. После этого мы присваиваем новые имена столбцов DataF
slide3_column2_pivot.columns = pd.MultiIndex.from_tuples([(col, 'slots_type' + str(i)) if col else (col, '') for i, col in enumerate(slide3_column2_pivot.columns)])

In [204]:
slide3_column2_pivot = slide3_column2_pivot.droplevel(0, axis=1)

In [205]:
slide3_column2_pivot

,,slots_type1,slots_type2,slots_type3,slots_type4,slots_type5,slots_type6
0,ГП1,3490,2153,5643,175,148,323
1,ГП12,7938,9163,17101,511,678,1189
2,ГП13,2741,1885,4626,225,145,370
3,ГП17,7560,4759,12319,1333,966,2299
4,ГП3,5435,4889,10324,448,496,944
5,ГП4,3436,2759,6195,400,247,647
6,ГП5,12324,6723,19047,1758,1196,2954
7,ГП6,3604,1561,5165,346,123,469
8,ГП8,4603,4666,9269,833,603,1436
9,ОБ11,3261,2875,6136,167,342,509


In [206]:
slide3_column2_pivot = slide3_column2_pivot.rename(columns={'': 'МО'})
slide3_column2_pivot = slide3_column2_pivot[['МО', 'slots_type4', 'slots_type3']]
slide3_column2_pivot = slide3_column2_pivot.rename_axis(None, axis=1)
slide3_column2_pivot['Доля неявок от общего числа записей'] = round(slide3_column2_pivot['slots_type4'] / slide3_column2_pivot['slots_type3'] * 100, 2)

In [207]:
slide3_column2_itog = slide3_column2_pivot[['МО', 'Доля неявок от общего числа записей']]

In [208]:
slide3_column2_itog

,МО,Доля неявок от общего числа записей
0,ГП1,3.10
1,ГП12,2.99
2,ГП13,4.86
3,ГП17,10.82
4,ГП3,4.34
5,ГП4,6.46
6,ГП5,9.23
7,ГП6,6.70
8,ГП8,8.99
9,ОБ11,2.72


3. Расчет третьего столбца 'Доля пациентов, принятых без предварительной записи'

In [209]:
indicators_slide3 = indi.copy()

In [210]:
indicators_slide3.rename(columns={'МО': 'Медицинская организация'}, inplace=True)

In [211]:
indicators_slide3 = indicators_slide3[['Медицинская организация', 
                    'Должность', 
                    'Общее число слотов в расписании', 
                    'Общее число записей', 
                    'из них вне слота с признаком "живая очередь"', 
                    'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени', 
                    'Общее число записей в расписании (занятые слоты)', 'из них из Прочие']].fillna(0)

In [212]:
indicators_slide3.columns

Index(['Медицинская организация', 'Должность',
       'Общее число слотов в расписании', 'Общее число записей',
       'из них вне слота с признаком "живая очередь"',
       'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени',
       'Общее число записей в расписании (занятые слоты)', 'из них из Прочие'],
      dtype='object')

In [213]:
indicators2 =  pd.merge(indicators_slide3, medorganization, how='left', on='Медицинская организация').dropna()

In [214]:
indicators2 = pd.merge(tablica_doljnost, indicators2, how='left', on='Должность').dropna()

In [215]:
indicators2

,Должность,Медицинская организация,Общее число слотов в расписании,Общее число записей,"из них вне слота с признаком ""живая очередь""",из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени,Общее число записей в расписании (занятые слоты),из них из Прочие,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,mo_short_name
0,врач-терапевт участковый,ГАУЗ ТО «ГП № 1»,2006.0,2434.0,23.0,708.0,2434.0,0.0,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»
1,врач-терапевт участковый,ГАУЗ ТО «ГП № 12»,5507.0,6606.0,300.0,1768.0,6606.0,0.0,ГАУЗ ТО «Городская поликлиника №12»,ГАУЗ ТО «ГП №12»,ГП12,"ГАУЗ ТО ""Городская поликлиника №12""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №12""",ГАУЗ ТО «Городская поликлиника № 12»
2,врач-терапевт участковый,ГАУЗ ТО «ГП № 13»,1873.0,2077.0,160.0,575.0,2077.0,0.0,ГАУЗ ТО «Городская поликлиника №13»,ГАУЗ ТО «ГП №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13
3,врач-терапевт участковый,ГАУЗ ТО «ГП №4»,1982.0,2204.0,70.0,783.0,2204.0,54.0,"ГАУЗ ТО ""Городская поликлиника №4""",ГАУЗ ТО «ГП №4»,ГП4,"ГАУЗ ТО ""Городская поликлиника №4""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №4""",ГАУЗ ТО «Городская поликлиника № 4»
4,врач-терапевт участковый,ГАУЗ ТО «ГП № 5»,6838.0,7644.0,192.0,1949.0,7644.0,21.0,"ГАУЗ ТО ""Городская поликлиника №5""",ГАУЗ ТО «ГП №5»,ГП5,"ГАУЗ ТО ""Городская поликлиника №5""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №5""",ГАУЗ ТО «Городская поликлиника № 5»
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,врач - детский хирург,"ГБУЗ ТО ""ОБ № 3""",205.0,249.0,0.0,54.0,249.0,0.0,ГБУЗ ТО «Областная больница №3» (г. Тобольск),ГБУЗ ТО «ОБ №3»,ОБ3,"ГБУЗ ТО ""Областная больница №3"" (г. Тобольск)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""(Г. ТОБОЛЬСК)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3"""
234,врач - детский хирург,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",48.0,61.0,1.0,15.0,61.0,0.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
235,врач - детский хирург,"ГБУЗ ТО ""ОКБ №1",0.0,1.0,1.0,0.0,1.0,0.0,ГБУЗ ТО «ОКБ №1»,ГБУЗ ТО «ОКБ №1»,ОКБ1,"ГБУЗ ТО ""ОКБ №1""",ГБУЗ ТО «ОКБ № 1»,ГБУЗ ТО «ОКБ №1»
236,врач - детский хирург,"ГБУЗ ТО ""ОКБ №2""",71.0,340.0,265.0,266.0,340.0,265.0,"ГБУЗ ТО ""ОКБ №2""",ГБУЗ ТО «ОКБ №2»,ОКБ2,"ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""Областная клиническая больница №2"""


In [216]:
# сопоставление 
indicator_mo_uchet_slide3 = pd.merge(indicators2, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [217]:
indicator_mo_uchet_slide3

,Должность,Медицинская организация,Общее число слотов в расписании,Общее число записей,"из них вне слота с признаком ""живая очередь""",из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени,Общее число записей в расписании (занятые слоты),из них из Прочие,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,mo_short_name,Проверка
0,врач-терапевт участковый,ГАУЗ ТО «ГП № 1»,2006.0,2434.0,23.0,708.0,2434.0,0.0,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «Городская поликлиника № 1»,1.0
1,врач-терапевт участковый,ГАУЗ ТО «ГП № 12»,5507.0,6606.0,300.0,1768.0,6606.0,0.0,ГАУЗ ТО «Городская поликлиника №12»,ГАУЗ ТО «ГП №12»,ГП12,"ГАУЗ ТО ""Городская поликлиника №12""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №12""",ГАУЗ ТО «Городская поликлиника № 12»,1.0
2,врач-терапевт участковый,ГАУЗ ТО «ГП № 13»,1873.0,2077.0,160.0,575.0,2077.0,0.0,ГАУЗ ТО «Городская поликлиника №13»,ГАУЗ ТО «ГП №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,1.0
3,врач-терапевт участковый,ГАУЗ ТО «ГП №4»,1982.0,2204.0,70.0,783.0,2204.0,54.0,"ГАУЗ ТО ""Городская поликлиника №4""",ГАУЗ ТО «ГП №4»,ГП4,"ГАУЗ ТО ""Городская поликлиника №4""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №4""",ГАУЗ ТО «Городская поликлиника № 4»,1.0
4,врач-терапевт участковый,ГАУЗ ТО «ГП № 5»,6838.0,7644.0,192.0,1949.0,7644.0,21.0,"ГАУЗ ТО ""Городская поликлиника №5""",ГАУЗ ТО «ГП №5»,ГП5,"ГАУЗ ТО ""Городская поликлиника №5""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №5""",ГАУЗ ТО «Городская поликлиника № 5»,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,врач - детский хирург,"ГБУЗ ТО ""ОБ № 3""",205.0,249.0,0.0,54.0,249.0,0.0,ГБУЗ ТО «Областная больница №3» (г. Тобольск),ГБУЗ ТО «ОБ №3»,ОБ3,"ГБУЗ ТО ""Областная больница №3"" (г. Тобольск)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""(Г. ТОБОЛЬСК)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""",1.0
234,врач - детский хирург,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",48.0,61.0,1.0,15.0,61.0,0.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.0
235,врач - детский хирург,"ГБУЗ ТО ""ОКБ №1",0.0,1.0,1.0,0.0,1.0,0.0,ГБУЗ ТО «ОКБ №1»,ГБУЗ ТО «ОКБ №1»,ОКБ1,"ГБУЗ ТО ""ОКБ №1""",ГБУЗ ТО «ОКБ № 1»,ГБУЗ ТО «ОКБ №1»,0.0
236,врач - детский хирург,"ГБУЗ ТО ""ОКБ №2""",71.0,340.0,265.0,266.0,340.0,265.0,"ГБУЗ ТО ""ОКБ №2""",ГБУЗ ТО «ОКБ №2»,ОКБ2,"ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""Областная клиническая больница №2""",1.0


In [218]:

slide3_column3_svod = pd.pivot_table(indicator_mo_uchet_slide3, index='МО', values=['Общее число слотов в расписании', 
                                                                      'Общее число записей', 
                                                                      'из них вне слота с признаком "живая очередь"', 
                                                                      'из них 2-я/3-я/4-я запись пациентов на один и тот же слот времени', 
                                                                      'Общее число записей в расписании (занятые слоты)'], 
                                                                      aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
slide3_column3_svod['Доля пациентов, принятых без предварительной записи'] = round(slide3_column3_svod['из них вне слота с признаком "живая очередь"'] / slide3_column3_svod['Общее число записей'] * 100, 2).fillna(0)
slide3_column3_svod = slide3_column3_svod[slide3_column3_svod['МО'] != 'ПЦ_Тюмень']
slide3_column3_svod = slide3_column3_svod[slide3_column3_svod['МО'] != 'ОКБ1']
slide3_column3 = slide3_column3_svod[['МО', 'Доля пациентов, принятых без предварительной записи']]
slide3_column3

,МО,"Доля пациентов, принятых без предварительной записи"
0,ГП1,3.14
1,ГП12,9.37
2,ГП13,11.24
3,ГП17,13.32
4,ГП3,13.22
5,ГП4,2.35
6,ГП5,3.06
7,ГП6,2.41
8,ГП8,18.68
9,ОБ11,10.54


4. Расчет показателя 'Доля посещений на дому'

In [219]:
def get_row_group_level(worksheet, row_num):
    cell = worksheet.cell(row=row_num, column=1)
    indentation_level = cell.alignment.indent
    return indentation_level

def count_row_group_levels(file_path):
    workbook = openpyxl.load_workbook(file_path)
    worksheet = workbook.active

    max_row = worksheet.max_row
    group_levels = {}

    for row_num in range(8, max_row + 1):
        group_level = get_row_group_level(worksheet, row_num)
        group_levels[row_num] = group_level

    return group_levels

folder_path = EMERGENCY_XLSX
combined_group_levels = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        row_group_levels = count_row_group_levels(file_path)
        dff = pd.DataFrame.from_dict(row_group_levels, orient='index', columns=['Уровень структуры'])
        dff['Файл'] = filename
        combined_group_levels.append(dff)

combined_df = pd.concat(combined_group_levels)
combined_df.index.name = 'Строка'
combined_df = combined_df.reset_index()
combined_df['Файл'] = combined_df['Файл'].apply(lambda x: x.split('.')[0])
clean_df = combined_df[['Уровень структуры']]
clean_df.loc[clean_df['Уровень структуры'] == 0, 'Уровень структуры'] = 1
clean_df

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\598535920.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.loc[clean_df['Уровень структуры'] == 0, 'Уровень структуры'] = 1


,Уровень структуры
0,1.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
3184,2.0
3185,2.0
3186,2.0
3187,2.0


In [220]:
emergency_medical_care = pd.read_excel(NEOTLOJKA_XLSX, skiprows=[0, 1, 2, 3, 4]).fillna(0)

In [221]:
emergency_medical_care = emergency_medical_care.rename(columns={'Записи на тип приема врача "Вызов на дом"': 'Метка "Неотложная помощь на дому"', 'Должность': 'Медицинская организация'})
emergency_medical_care.drop(index=0, inplace=True)

In [222]:
emergency_medical_care.reset_index(drop=True, inplace=True)

In [223]:
emergency_medical_care = emergency_medical_care[['Медицинская организация', 'Метка "Неотложная помощь на дому"', 'Метка "Неотложная помощь в поликлинике"']]

In [224]:
emergency_medical_care_merge = pd.concat([emergency_medical_care, clean_df], axis=1)

In [225]:
emergency_medical_care_merge['МО'] = np.where(emergency_medical_care_merge['Уровень структуры'] == 1, emergency_medical_care_merge['Медицинская организация'], np.NaN)
emergency_medical_care_merge['МО'] = emergency_medical_care_merge['МО'].ffill()

In [226]:
emergency_medical_care_merge = emergency_medical_care_merge[emergency_medical_care_merge['Уровень структуры'] != 1.0]

In [227]:
emergency_medical_care_merge = emergency_medical_care_merge[(emergency_medical_care_merge['Медицинская организация'] == 'врач-педиатр участковый') | (emergency_medical_care_merge['Медицинская организация'] == 'врач-терапевт участковый')]

In [228]:
emergency_medical_care_merge = emergency_medical_care_merge[['МО', 'Метка "Неотложная помощь на дому"', 'Метка "Неотложная помощь в поликлинике"']]

In [229]:
emergency_medical_care_merge

,МО,"Метка ""Неотложная помощь на дому""","Метка ""Неотложная помощь в поликлинике"""
5,"ГАУЗ ТО ""Городская поликлиника № 1""",0,0.0
7,"ГАУЗ ТО ""Городская поликлиника № 1""",0,0.0
17,"ГАУЗ ТО ""Городская поликлиника № 1""",1,4.0
21,"ГАУЗ ТО ""Городская поликлиника № 1""",0,0.0
37,"ГАУЗ ТО ""Городская поликлиника № 1""",0,0.0
...,...,...,...
3129,"Тюменская больница ФГБУЗ ""ЗСМЦ ФМБА России""",0,0.0
3130,"Тюменская больница ФГБУЗ ""ЗСМЦ ФМБА России""",0,0.0
3139,"Тюменская больница ФГБУЗ ""ЗСМЦ ФМБА России""",0,36.0
3171,"Тюменская больница ФГБУЗ ""ЗСМЦ ФМБА России""",0,4.0


In [230]:
emergency_medical_care_merge.to_excel('C:\\Users\\propp_sa\\Desktop\\VScode\\prezentation\\output\\неотложка.xlsx', index=False)

In [231]:
emergency_medical_care_pivot1 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь на дому"'], aggfunc=np.sum).reset_index()
emergency_medical_care_pivot2 = pd.pivot_table(emergency_medical_care_merge, index='МО', values=['Метка "Неотложная помощь в поликлинике"'], aggfunc=np.sum).reset_index()
emergency_medical_care_pivot_merge2 = pd.merge(emergency_medical_care_pivot1, emergency_medical_care_pivot2, how='inner', on='МО')
emergency_medical_care_pivot_merge2['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] = emergency_medical_care_pivot_merge2['Метка "Неотложная помощь на дому"'] + emergency_medical_care_pivot_merge2['Метка "Неотложная помощь в поликлинике"']
emergency_medical_care_pivot_merge2.rename(columns={'МО' : 'mo_short_name'}, inplace=True)
emergency_medical_care_pivot_merge2

,mo_short_name,"Метка ""Неотложная помощь на дому""","Метка ""Неотложная помощь в поликлинике""",Сумма посещений на дому по неотложной помощи у врачей участковой службы
0,"ГАУЗ ТО ""Городская поликлиника № 1""",202,109.0,311.0
1,"ГАУЗ ТО ""Городская поликлиника № 12""",0,39.0,39.0
2,"ГАУЗ ТО ""Городская поликлиника № 13 """,0,0.0,0.0
3,"ГАУЗ ТО ""Городская поликлиника № 4""",208,181.0,389.0
4,"ГАУЗ ТО ""Городская поликлиника №5""",0,0.0,0.0
5,"ГАУЗ ТО ""Областная больница № 19"" (г.Тюмень)",4,20.0,24.0
6,ГАУЗ ТО «Городская Поликлиника № 6»,57,0.0,57.0
7,ГАУЗ ТО «Городская поликлиника № 8»,80,0.0,80.0
8,ГАУЗ ТО «Городская поликлиника №17»,0,4.0,4.0
9,ГАУЗ ТО «Городская поликлиника №3»,0,0.0,0.0


In [232]:
merge_ras_1c_uchet

,Организация,mp_dolgnost,Проверка,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,...,slots_created,slots_booked,slots_free_after_booked,visits_absence,visits_success,Наименование МО,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
10,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
11,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,6,6,0,0,6,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
12,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
13,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,7,7,0,0,7,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
14,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,4,4,0,0,4,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87455,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,12,12,0,0,12,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87456,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87457,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,11,11,0,0,11,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87458,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""


###### Загружаем секретный отчет

In [233]:
merge_ras_1c_uchet.columns

Index(['Организация', 'mp_dolgnost', 'Проверка', 'region', 'mo_oid',
       'mo_short_name', 'sp_oid', 'sp_name', 'mp_id', 'mp_fio', 'date_report',
       'mp_stavka', 'slots_type', 'slots_referral', 'slot_lenght',
       'slots_created', 'slots_booked', 'slots_free_after_booked',
       'visits_absence', 'visits_success', 'Наименование МО', 'МО',
       'МедОрганизация', 'Краткое наименование организации',
       'Медицинская организация'],
      dtype='object')

In [234]:
merge_ras_1c_uchet_secret = merge_ras_1c_uchet.copy()

In [235]:
merge_ras_1c_uchet_secret.rename(columns={'mp_dolgnost' : 'Должность'}, inplace=True)

In [236]:
merge_ras_1c_uchet_secret['date_report'].unique()

array(['2024-02-19T00:00:00.000000000', '2024-02-20T00:00:00.000000000',
       '2024-02-21T00:00:00.000000000', '2024-02-22T00:00:00.000000000',
       '2024-02-24T00:00:00.000000000', '2024-02-23T00:00:00.000000000',
       '2024-02-25T00:00:00.000000000'], dtype='datetime64[ns]')

In [237]:
uchastkovye_secret = merge_ras_1c_uchet_secret[(merge_ras_1c_uchet_secret['Должность'] == 'врач-педиатр участковый') | (merge_ras_1c_uchet_secret['Должность'] == 'врач-терапевт участковый')]

In [238]:
uchastkovye_secret

,Организация,Должность,Проверка,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,...,slots_created,slots_booked,slots_free_after_booked,visits_absence,visits_success,Наименование МО,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
8533,ГАУЗ ТО «ГП №1»,врач-терапевт участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7315,ГАУЗ ТО «Городская поликлиника № 1»,1.2.643.5.1.13.13.12.2.72.7315.0.12216,1 Терапия,15037816048,Азизов Руслан Азизович,...,9,9,0,0,9,ГАУЗ ТО «Городская поликлиника №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»
8534,ГАУЗ ТО «ГП №1»,врач-терапевт участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7315,ГАУЗ ТО «Городская поликлиника № 1»,1.2.643.5.1.13.13.12.2.72.7315.0.12216,1 Терапия,15037816048,Азизов Руслан Азизович,...,3,3,0,0,3,ГАУЗ ТО «Городская поликлиника №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»
8535,ГАУЗ ТО «ГП №1»,врач-терапевт участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7315,ГАУЗ ТО «Городская поликлиника № 1»,1.2.643.5.1.13.13.12.2.72.7315.0.12216,1 Терапия,15037816048,Азизов Руслан Азизович,...,21,21,0,0,21,ГАУЗ ТО «Городская поликлиника №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»
8536,ГАУЗ ТО «ГП №1»,врач-терапевт участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7315,ГАУЗ ТО «Городская поликлиника № 1»,1.2.643.5.1.13.13.12.2.72.7315.0.12216,1 Терапия,15037816048,Азизов Руслан Азизович,...,3,2,0,0,2,ГАУЗ ТО «Городская поликлиника №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»
8537,ГАУЗ ТО «ГП №1»,врач-терапевт участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7315,ГАУЗ ТО «Городская поликлиника № 1»,1.2.643.5.1.13.13.12.2.72.7315.0.12216,1 Терапия,15037816048,Азизов Руслан Азизович,...,6,2,0,1,1,ГАУЗ ТО «Городская поликлиника №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70819,ГАУЗ ТО «ГП №13»,врач-педиатр участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7321,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,1.2.643.5.1.13.13.12.2.72.7321.0.334367,Консульт.диагн.отд,5419083351,Федоткина Галина Федоровна,...,3,3,0,1,2,ГАУЗ ТО «Городская поликлиника №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»
70820,ГАУЗ ТО «ГП №13»,врач-педиатр участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7321,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,1.2.643.5.1.13.13.12.2.72.7321.0.334367,Консульт.диагн.отд,5419083351,Федоткина Галина Федоровна,...,4,3,0,0,3,ГАУЗ ТО «Городская поликлиника №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»
70821,ГАУЗ ТО «ГП №13»,врач-педиатр участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7321,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,1.2.643.5.1.13.13.12.2.72.7321.0.334367,Консульт.диагн.отд,5419083351,Федоткина Галина Федоровна,...,6,6,0,0,0,ГАУЗ ТО «Городская поликлиника №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»
70822,ГАУЗ ТО «ГП №13»,врач-педиатр участковый,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7321,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,1.2.643.5.1.13.13.12.2.72.7321.0.334367,Консульт.диагн.отд,5419083351,Федоткина Галина Федоровна,...,5,5,0,0,0,ГАУЗ ТО «Городская поликлиника №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»


In [239]:
uchastkovye_secret['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый'],
      dtype=object)

In [240]:
uchastkovye_secret['count_number'] = 1
uchastkovye_secret_pivot = pd.pivot_table(uchastkovye_secret, index=['МО'], columns='Должность', values='count_number', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
uchastkovye_secret_pivot.rename_axis(None, axis=1, inplace=True)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1098382493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uchastkovye_secret['count_number'] = 1


In [241]:
uchastkovye_secret_pivot

,МО,врач-педиатр участковый,врач-терапевт участковый,Итого
0,ГП1,227,239,466
1,ГП12,556,813,1369
2,ГП13,166,239,405
3,ГП17,508,515,1023
4,ГП3,331,841,1172
5,ГП4,306,259,565
6,ГП5,633,1047,1680
7,ГП6,234,427,661
8,ГП8,342,417,759
9,ОБ11,217,338,555


In [242]:

uchastkovye_secret_pivot['Участковые'] = uchastkovye_secret_pivot['врач-педиатр участковый'] + uchastkovye_secret_pivot['врач-терапевт участковый']

In [243]:
uchastkovye_secret_pivot

,МО,врач-педиатр участковый,врач-терапевт участковый,Итого,Участковые
0,ГП1,227,239,466,466
1,ГП12,556,813,1369,1369
2,ГП13,166,239,405,405
3,ГП17,508,515,1023,1023
4,ГП3,331,841,1172,1172
5,ГП4,306,259,565,565
6,ГП5,633,1047,1680,1680
7,ГП6,234,427,661,661
8,ГП8,342,417,759,759
9,ОБ11,217,338,555,555


In [244]:
uchastkovye_itog = uchastkovye_secret_pivot[['МО', 'Участковые']]

In [245]:
uchastkovye_itog['МО'].unique()

array(['ГП1', 'ГП12', 'ГП13', 'ГП17', 'ГП3', 'ГП4', 'ГП5', 'ГП6', 'ГП8',
       'ОБ11', 'ОБ12', 'ОБ13', 'ОБ14', 'ОБ15', 'ОБ19', 'ОБ20', 'ОБ23',
       'ОБ24', 'ОБ3', 'ОБ4', 'ОБ9', 'ОКБ2', 'Итого'], dtype=object)

In [246]:
guide_mo2 = pd.read_excel(ROOT_PATH +'\\guide\\guide_mo2.xlsx')

In [247]:
guide_mo2

,mo_short_name,МО
0,"ГАУЗ ТО ""Городская поликлиника № 1""",ГП1
1,"ГАУЗ ТО ""Городская поликлиника № 12""",ГП12
2,"ГАУЗ ТО ""Городская поликлиника № 13 """,ГП13
3,"ГАУЗ ТО ""Городская поликлиника № 4""",ГП4
4,"ГАУЗ ТО ""Городская поликлиника №5""",ГП5
5,"ГАУЗ ТО ""Областная больница № 19"" (г.Тюмень)",ОБ19
6,ГАУЗ ТО «Городская Поликлиника № 6»,ГП6
7,ГАУЗ ТО «Городская поликлиника № 8»,ГП8
8,ГАУЗ ТО «Городская поликлиника №17»,ГП17
9,ГАУЗ ТО «Городская поликлиника №3»,ГП3


In [248]:
emergency_medical_guide = pd.merge(guide_mo2, emergency_medical_care_pivot_merge2, on='mo_short_name', how='right')

In [249]:
emergency_medical_guide

,mo_short_name,МО,"Метка ""Неотложная помощь на дому""","Метка ""Неотложная помощь в поликлинике""",Сумма посещений на дому по неотложной помощи у врачей участковой службы
0,"ГАУЗ ТО ""Городская поликлиника № 1""",ГП1,202,109.0,311.0
1,"ГАУЗ ТО ""Городская поликлиника № 12""",ГП12,0,39.0,39.0
2,"ГАУЗ ТО ""Городская поликлиника № 13 """,ГП13,0,0.0,0.0
3,"ГАУЗ ТО ""Городская поликлиника № 4""",ГП4,208,181.0,389.0
4,"ГАУЗ ТО ""Городская поликлиника №5""",ГП5,0,0.0,0.0
5,"ГАУЗ ТО ""Областная больница № 19"" (г.Тюмень)",ОБ19,4,20.0,24.0
6,ГАУЗ ТО «Городская Поликлиника № 6»,ГП6,57,0.0,57.0
7,ГАУЗ ТО «Городская поликлиника № 8»,ГП8,80,0.0,80.0
8,ГАУЗ ТО «Городская поликлиника №17»,ГП17,0,4.0,4.0
9,ГАУЗ ТО «Городская поликлиника №3»,ГП3,0,0.0,0.0


###### Объединяеем план и факт расчета по участковой службе

In [250]:
emergency_medical_guide_itog = emergency_medical_guide[['МО', 'Сумма посещений на дому  по неотложной помощи у врачей участковой службы']]

In [251]:
emergency_medical_guide_itog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   МО                                                                        20 non-null     object 
 1   Сумма посещений на дому  по неотложной помощи у врачей участковой службы  21 non-null     float64
dtypes: float64(1), object(1)
memory usage: 504.0+ bytes


In [252]:
itog_columns = 'Итого'

In [253]:
sum_columns = emergency_medical_guide_itog['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'].sum()

In [254]:
sum_columns

1394.0

In [255]:
new_row = {'МО': itog_columns, 'Сумма посещений на дому  по неотложной помощи у врачей участковой службы': sum_columns}

In [256]:
emergency_medical_guide_itog = emergency_medical_guide_itog.append(new_row, ignore_index=True)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\713752448.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emergency_medical_guide_itog = emergency_medical_guide_itog.append(new_row, ignore_index=True)


In [257]:
emergency_medical_guide_itog

,МО,Сумма посещений на дому по неотложной помощи у врачей участковой службы
0,ГП1,311.0
1,ГП12,39.0
2,ГП13,0.0
3,ГП4,389.0
4,ГП5,0.0
5,ОБ19,24.0
6,ГП6,57.0
7,ГП8,80.0
8,ГП17,4.0
9,ГП3,0.0


In [258]:
merge_emergency_uchastkov = pd.merge(emergency_medical_guide_itog, uchastkovye_itog, on='МО', how='outer')

In [259]:
merge_emergency_uchastkov['Доля посещений на дому'] = np.where(merge_emergency_uchastkov['Участковые'] != 0 & (merge_emergency_uchastkov['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] != 0), round(merge_emergency_uchastkov['Сумма посещений на дому  по неотложной помощи у врачей участковой службы'] / merge_emergency_uchastkov['Участковые'] * 100, 1), 0)

In [260]:
merge_emergency_uchastkov

,МО,Сумма посещений на дому по неотложной помощи у врачей участковой службы,Участковые,Доля посещений на дому
0,ГП1,311.0,466.0,66.7
1,ГП12,39.0,1369.0,2.8
2,ГП13,0.0,405.0,0.0
3,ГП4,389.0,565.0,68.8
4,ГП5,0.0,1680.0,0.0
5,ОБ19,24.0,1245.0,1.9
6,ГП6,57.0,661.0,8.6
7,ГП8,80.0,759.0,10.5
8,ГП17,4.0,1023.0,0.4
9,ГП3,0.0,1172.0,0.0


In [261]:
slide_3_column_4_itog = merge_emergency_uchastkov[['МО', 'Доля посещений на дому']]

In [262]:
med_slide3 = medorganization[['МО', 'Организация']]

In [263]:
row_to_move2 = med_slide3[med_slide3['МО'] == 'ОКБ1']

In [264]:
row_to_move2

,МО,Организация
28,ОКБ1,ГБУЗ ТО «ОКБ №1»


In [265]:
med_slide3 = med_slide3.drop(row_to_move2.index)

In [266]:
row_to_move3 = med_slide3[med_slide3['МО'] == 'ПЦ_Тюмень']

In [267]:
med_slide3 = med_slide3.drop(row_to_move3.index)

In [268]:
med_slide3

,МО,Организация
0,ОСП,"ГАУЗ ТО ""ОСП"""
1,ГП1,ГАУЗ ТО «ГП №1»
2,ГП12,ГАУЗ ТО «ГП №12»
3,ГП13,ГАУЗ ТО «ГП №13»
4,ГП17,ГАУЗ ТО «ГП №17»
5,ГП3,ГАУЗ ТО «ГП №3»
6,ГП4,ГАУЗ ТО «ГП №4»
7,ГП6,ГАУЗ ТО «ГП №6»
8,ГП5,ГАУЗ ТО «ГП №5»
9,ГП8,ГАУЗ ТО «ГП №8»


In [269]:
med_slide3 = med_slide3.append({'МО': 'Итого', 'Организация': 'Итого'}, ignore_index=True)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\3132241464.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  med_slide3 = med_slide3.append({'МО': 'Итого', 'Организация': 'Итого'}, ignore_index=True)


In [270]:
data_frames3 = [slide_3_column_1_itog, slide3_column2_itog, slide3_column3, slide_3_column_4_itog]

In [271]:
pivot_ras_1c_s3 = reduce(lambda  left,right: pd.merge(left,right,on=['МО'], how='outer'), data_frames3)

In [272]:
pivot_ras_1c_s3

,МО,Занятость слотов всего (доля от общего числа созданных и количества занятых),Доля неявок от общего числа записей,"Доля пациентов, принятых без предварительной записи",Доля посещений на дому
0,ГП1,81.14,3.10,3.14,66.7
1,ГП12,89.43,2.99,9.37,2.8
2,ГП13,81.73,4.86,11.24,0.0
3,ГП17,91.43,10.82,13.32,0.4
4,ГП3,86.24,4.34,13.22,0.0
5,ГП4,85.59,6.46,2.35,68.8
6,ГП5,85.64,9.23,3.06,0.0
7,ГП6,76.43,6.70,2.41,8.6
8,ГП8,74.11,8.99,18.68,10.5
9,ОБ11,73.25,2.72,10.54,0.9


In [273]:
pivot_ras_1c_s3 = pivot_ras_1c_s3.dropna(subset=['МО'])

In [274]:
result_slide3 = pd.merge(pivot_ras_1c_s3, med_slide3, how='left', on='МО')

In [275]:
result_slide3 = result_slide3[['Организация', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)', 'Доля неявок от общего числа записей', 'Доля пациентов, принятых без предварительной записи', 'Доля посещений на дому']]

In [276]:
result_slide3 

,Организация,Занятость слотов всего (доля от общего числа созданных и количества занятых),Доля неявок от общего числа записей,"Доля пациентов, принятых без предварительной записи",Доля посещений на дому
0,ГАУЗ ТО «ГП №1»,81.14,3.10,3.14,66.7
1,ГАУЗ ТО «ГП №12»,89.43,2.99,9.37,2.8
2,ГАУЗ ТО «ГП №13»,81.73,4.86,11.24,0.0
3,ГАУЗ ТО «ГП №17»,91.43,10.82,13.32,0.4
4,ГАУЗ ТО «ГП №3»,86.24,4.34,13.22,0.0
5,ГАУЗ ТО «ГП №4»,85.59,6.46,2.35,68.8
6,ГАУЗ ТО «ГП №5»,85.64,9.23,3.06,0.0
7,ГАУЗ ТО «ГП №6»,76.43,6.70,2.41,8.6
8,ГАУЗ ТО «ГП №8»,74.11,8.99,18.68,10.5
9,ГБУЗ ТО «ОБ №11»,73.25,2.72,10.54,0.9


In [277]:
# Создаем DataFrame
result_slide3.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide3.fillna(0, inplace=True)
df3 = result_slide3

In [278]:
df3

,Наименование МО,Занятость слотов всего (доля от общего числа созданных и количества занятых),Доля неявок от общего числа записей,"Доля пациентов, принятых без предварительной записи",Доля посещений на дому
0,ГАУЗ ТО «ГП №1»,81.14,3.10,3.14,66.7
1,ГАУЗ ТО «ГП №12»,89.43,2.99,9.37,2.8
2,ГАУЗ ТО «ГП №13»,81.73,4.86,11.24,0.0
3,ГАУЗ ТО «ГП №17»,91.43,10.82,13.32,0.4
4,ГАУЗ ТО «ГП №3»,86.24,4.34,13.22,0.0
5,ГАУЗ ТО «ГП №4»,85.59,6.46,2.35,68.8
6,ГАУЗ ТО «ГП №5»,85.64,9.23,3.06,0.0
7,ГАУЗ ТО «ГП №6»,76.43,6.70,2.41,8.6
8,ГАУЗ ТО «ГП №8»,74.11,8.99,18.68,10.5
9,ГБУЗ ТО «ОБ №11»,73.25,2.72,10.54,0.9


##### Создаем слайд 4

1. Слайд 4 Таблица 1 Столбец 1

In [279]:
# Подгружаем записи
record = indi


In [280]:
record.rename(columns={'МО': 'Медицинская организация'}, inplace=True)

In [281]:
record_merge = pd.merge(medorganization, record, on='Медицинская организация', how='left')

In [282]:
tablica_doljnost

,Должность
0,врач-терапевт участковый
1,врач-педиатр участковый
2,врач общей практики (семейный врач)
3,врач-офтальмолог
4,врач-оториноларинголог
5,врач-хирург
6,врач-акушер-гинеколог
7,врач-психиатр-нарколог
8,врач-фтизиатр участковый
9,врач-фтизиатр


In [283]:
record_merge['Должность'].unique()

array(['врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог-хирург', 'зубной врач', 'врач - детский хирург',
       'врач общей практики (семейный врач)',
       'врач по медицинской профилактике', 'врач-акушер-гинеколог',
       'врач-оториноларинголог', 'врач-офтальмолог', 'врач-педиатр',
       'врач-педиатр участковый', 'врач-терапевт',
       'врач-терапевт участковый', 'врач-уролог', 'врач-хирург',
       'фельдшер', nan, 'врач-психиатр детский',
       'врач-стоматолог детский', 'врач-психиатр-нарколог',
       'врач-фтизиатр', 'врач-психиатр детский участковый',
       'врач-фтизиатр участковый'], dtype=object)

In [284]:
record_merge_doljnost = pd.merge(tablica_doljnost, record_merge, how='left', on='Должность')

In [285]:
record_merge_doljnost['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый'],
      dtype=object)

In [286]:
record_merge_doljnost

,Должность,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,Медицинская организация,mo_short_name,Unnamed: 1,Unnamed: 2,...,Число поступивших обращений с целью записи на прием к врачу-специалисту по поводу заболеваний повторная,Число поступивших обращений с целью записи на прием к врачу-специалисту в рамках диспансерного наблюдения,Число поступивших обращений с целью записи на услуги инструментальной и/или лабораторной диагностики,"Число поступивших обращений с целью получения назначения, направления, справки, выписки, листа нетрудоспособности, иных документов",Число поступивших обращений с целью записи для прохождения первого или второго этапа диспансеризации,Число поступивших обращений с целью записи на вакцинацию,Число операторов колл-центров,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Число звонков для обратной связи (уточнение цели обращения и актуализация записи)
0,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»,ГАУЗ ТО «Городская поликлиника № 1»,NaN,NaN,...,603.0,13.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN
1,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №12»,ГАУЗ ТО «ГП №12»,ГП12,"ГАУЗ ТО ""Городская поликлиника №12""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №12""",ГАУЗ ТО «ГП № 12»,ГАУЗ ТО «Городская поликлиника № 12»,NaN,NaN,...,2285.0,256.0,NaN,1.0,NaN,NaN,NaN,4.0,2.0,NaN
2,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №13»,ГАУЗ ТО «ГП №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,NaN,NaN,...,240.0,145.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №17»,ГАУЗ ТО «ГП №17»,ГП17,"ГАУЗ ТО ""Городская поликлиника №17""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА № 17""",ГАУЗ ТО «ГП № 17»,ГАУЗ ТО «Городская поликлиника № 17»,NaN,NaN,...,571.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN
4,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №3»,ГАУЗ ТО «ГП №3»,ГП3,"ГАУЗ ТО ""Городская поликлиника №3""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №3""",ГАУЗ ТО «ГП № 3»,ГАУЗ ТО «Городская поликлиника № 3»,NaN,NaN,...,431.0,18.0,NaN,NaN,366.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,врач - детский хирург,ГБУЗ ТО «Областная больница №3» (г. Тобольск),ГБУЗ ТО «ОБ №3»,ОБ3,"ГБУЗ ТО ""Областная больница №3"" (г. Тобольск)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""(Г. ТОБОЛЬСК)","ГБУЗ ТО ""ОБ № 3""","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""",NaN,NaN,...,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,врач - детский хирург,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)",NaN,NaN,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,врач - детский хирург,"ГБУЗ ТО ""ОКБ №2""",ГБУЗ ТО «ОКБ №2»,ОКБ2,"ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""Областная клиническая больница №2""",NaN,NaN,...,22.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,врач - детский хирург,ГБУЗ ТО «ОКБ №1»,ГБУЗ ТО «ОКБ №1»,ОКБ1,"ГБУЗ ТО ""ОКБ №1""",ГБУЗ ТО «ОКБ № 1»,"ГБУЗ ТО ""ОКБ №1",ГБУЗ ТО «ОКБ №1»,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [287]:
record_merge_uchet = pd.merge(record_merge_doljnost, employee_uchet, on=['Организация', 'Должность'], how='left').fillna(0)

In [288]:
record_merge_uchet

,Должность,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,Медицинская организация,mo_short_name,Unnamed: 1,Unnamed: 2,...,Число поступивших обращений с целью записи на прием к врачу-специалисту в рамках диспансерного наблюдения,Число поступивших обращений с целью записи на услуги инструментальной и/или лабораторной диагностики,"Число поступивших обращений с целью получения назначения, направления, справки, выписки, листа нетрудоспособности, иных документов",Число поступивших обращений с целью записи для прохождения первого или второго этапа диспансеризации,Число поступивших обращений с целью записи на вакцинацию,Число операторов колл-центров,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Число звонков для обратной связи (уточнение цели обращения и актуализация записи),Проверка
0,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №1»,ГАУЗ ТО «ГП №1»,ГП1,"ГАУЗ ТО ""Городская поликлиника №1""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №1""",ГАУЗ ТО «ГП № 1»,ГАУЗ ТО «Городская поликлиника № 1»,0.0,0.0,...,13.0,0.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №12»,ГАУЗ ТО «ГП №12»,ГП12,"ГАУЗ ТО ""Городская поликлиника №12""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №12""",ГАУЗ ТО «ГП № 12»,ГАУЗ ТО «Городская поликлиника № 12»,0.0,0.0,...,256.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0
2,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №13»,ГАУЗ ТО «ГП №13»,ГП13,"ГАУЗ ТО ""Городская поликлиника №13""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №13""",ГАУЗ ТО «ГП № 13»,ГАУЗ ТО ГОРОДСКАЯ ПОЛИКЛИНИКА 13,0.0,0.0,...,145.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №17»,ГАУЗ ТО «ГП №17»,ГП17,"ГАУЗ ТО ""Городская поликлиника №17""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА № 17""",ГАУЗ ТО «ГП № 17»,ГАУЗ ТО «Городская поликлиника № 17»,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0
4,врач-терапевт участковый,ГАУЗ ТО «Городская поликлиника №3»,ГАУЗ ТО «ГП №3»,ГП3,"ГАУЗ ТО ""Городская поликлиника №3""","ГАУЗ ТО ""ГОРОДСКАЯ ПОЛИКЛИНИКА №3""",ГАУЗ ТО «ГП № 3»,ГАУЗ ТО «Городская поликлиника № 3»,0.0,0.0,...,18.0,0.0,0.0,366.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,врач - детский хирург,ГБУЗ ТО «Областная больница №3» (г. Тобольск),ГБУЗ ТО «ОБ №3»,ОБ3,"ГБУЗ ТО ""Областная больница №3"" (г. Тобольск)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""(Г. ТОБОЛЬСК)","ГБУЗ ТО ""ОБ № 3""","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА № 3""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
234,врач - детский хирург,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
235,врач - детский хирург,"ГБУЗ ТО ""ОКБ №2""",ГБУЗ ТО «ОКБ №2»,ОКБ2,"ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""ОКБ №2""","ГБУЗ ТО ""Областная клиническая больница №2""",0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236,врач - детский хирург,ГБУЗ ТО «ОКБ №1»,ГБУЗ ТО «ОКБ №1»,ОКБ1,"ГБУЗ ТО ""ОКБ №1""",ГБУЗ ТО «ОКБ № 1»,"ГБУЗ ТО ""ОКБ №1",ГБУЗ ТО «ОКБ №1»,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
include_to_list = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц включенных в лист ожидания', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
include_to_list

,МО,Число лиц включенных в лист ожидания
0,ГП1,197.0
1,ГП12,246.0
2,ГП13,10.0
3,ГП17,326.0
4,ГП3,349.0
5,ГП4,54.0
6,ГП5,743.0
7,ГП6,6.0
8,ГП8,383.0
9,ОБ11,27.0


2. Слайд 4 Таблица 1 Столбец 2

In [290]:
recorded = pd.pivot_table(record_merge_uchet, index='МО', values='Число лиц исключенных из листа ожидания по причине записи', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
recorded

,МО,Число лиц исключенных из листа ожидания по причине записи
0,ГП1,43.0
1,ГП12,103.0
2,ГП13,4.0
3,ГП17,115.0
4,ГП3,67.0
5,ГП4,44.0
6,ГП5,364.0
7,ГП6,4.0
8,ГП8,24.0
9,ОБ11,10.0


3. Слайд 4 Таблица 1 Столбец 3

In [291]:
competitive = pd.pivot_table(record_merge_uchet, index='МО', values='из них конкурентных слотов', aggfunc=np.sum, margins=True, margins_name='Итого').reset_index()
competitive.rename(columns={'из них конкурентных слотов' : 'Количество конкурентной записи'}, inplace=True)

In [292]:
competitive

,МО,Количество конкурентной записи
0,ГП1,3841.0
1,ГП12,8696.0
2,ГП13,3134.0
3,ГП17,7614.0
4,ГП3,6068.0
5,ГП4,3304.0
6,ГП5,11004.0
7,ГП6,4159.0
8,ГП8,6784.0
9,ОБ11,4040.0


In [293]:
data_frames4 = [include_to_list, recorded, competitive]
slide4_df = reduce(lambda  left,right: pd.merge(left,right,on=['МО'], how='outer'), data_frames4)

###### Слайд 4 столбец 4

In [294]:
slide4_df['Доля лиц, включенных в лист ожидания от общего количества конкурентной записи'] = round((slide4_df['Число лиц включенных в лист ожидания'] / slide4_df['Количество конкурентной записи']) * 100, 2).fillna(0)
slide4_df['Движение лиц в листе ожидания'] = round((slide4_df['Число лиц включенных в лист ожидания'] - slide4_df['Число лиц исключенных из листа ожидания по причине записи']) / slide4_df['Число лиц включенных в лист ожидания'] * 100, 2).fillna(0)

###### Слайд 4 столбец 5

In [295]:
slide4_df

,МО,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Количество конкурентной записи,"Доля лиц, включенных в лист ожидания от общего количества конкурентной записи",Движение лиц в листе ожидания
0,ГП1,197.0,43.0,3841.0,5.13,78.17
1,ГП12,246.0,103.0,8696.0,2.83,58.13
2,ГП13,10.0,4.0,3134.0,0.32,60.00
3,ГП17,326.0,115.0,7614.0,4.28,64.72
4,ГП3,349.0,67.0,6068.0,5.75,80.80
5,ГП4,54.0,44.0,3304.0,1.63,18.52
6,ГП5,743.0,364.0,11004.0,6.75,51.01
7,ГП6,6.0,4.0,4159.0,0.14,33.33
8,ГП8,383.0,24.0,6784.0,5.65,93.73
9,ОБ11,27.0,10.0,4040.0,0.67,62.96


In [296]:
slide4_df.sort_values(by='Движение лиц в листе ожидания', ascending=False, inplace=True)

In [297]:
slide4_df.reset_index(drop=True, inplace=True)

In [298]:
# Получаем индекс строки, которую нужно переместить в конец
index_itog_slide4 = slide4_df.index[slide4_df['МО'] == 'Итого'].tolist()[0]
row_to_move = slide4_df.loc[index_itog_slide4]
slide4_df = slide4_df.drop(index_itog_slide4)
# Добавляем извлеченную строку в конец DataFrame
slide4_df = slide4_df.append(row_to_move)

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\1434060210.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  slide4_df = slide4_df.append(row_to_move)


In [299]:
slide4_df.reset_index(drop=True, inplace=True)

In [300]:
slide4_df

,МО,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Количество конкурентной записи,"Доля лиц, включенных в лист ожидания от общего количества конкурентной записи",Движение лиц в листе ожидания
0,ОБ15,11.0,0.0,1450.0,0.76,100.00
1,ОКБ1,5.0,0.0,0.0,inf,100.00
2,ОБ9,2.0,0.0,1198.0,0.17,100.00
3,ОБ19,65.0,3.0,6502.0,1.00,95.38
4,ГП8,383.0,24.0,6784.0,5.65,93.73
5,ОБ4,91.0,7.0,7539.0,1.21,92.31
6,ОБ12,9.0,1.0,4172.0,0.22,88.89
7,ОБ24,25.0,3.0,1193.0,2.10,88.00
8,ОБ23,96.0,12.0,3030.0,3.17,87.50
9,ОКБ2,63.0,8.0,2628.0,2.40,87.30


In [301]:
# Создаем DataFrame
result_slide4 = pd.merge(slide4_df, med_slide3, how='left', on='МО')
result_slide4 = result_slide4[['Организация', 'Число лиц включенных в лист ожидания', 'Число лиц исключенных из листа ожидания по причине записи', 'Количество конкурентной записи', 'Доля лиц, включенных в лист ожидания от общего количества конкурентной записи', 'Движение лиц в листе ожидания']]
result_slide4.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
result_slide4.fillna(0, inplace=True)
df4 = result_slide4

In [302]:
df4

,Наименование МО,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Количество конкурентной записи,"Доля лиц, включенных в лист ожидания от общего количества конкурентной записи",Движение лиц в листе ожидания
0,ГБУЗ ТО «ОБ №15»,11.0,0.0,1450.0,0.76,100.00
1,0,5.0,0.0,0.0,inf,100.00
2,ГБУЗ ТО «ОБ №9»,2.0,0.0,1198.0,0.17,100.00
3,ГБУЗ ТО «ОБ №19»,65.0,3.0,6502.0,1.00,95.38
4,ГАУЗ ТО «ГП №8»,383.0,24.0,6784.0,5.65,93.73
5,ГБУЗ ТО «ОБ №4»,91.0,7.0,7539.0,1.21,92.31
6,ГБУЗ ТО «ОБ №12»,9.0,1.0,4172.0,0.22,88.89
7,ГБУЗ ТО «ОБ №24»,25.0,3.0,1193.0,2.10,88.00
8,ГБУЗ ТО «ОБ №23»,96.0,12.0,3030.0,3.17,87.50
9,ГБУЗ ТО «ОКБ №2»,63.0,8.0,2628.0,2.40,87.30


##### Слайд 5
* Обрабатываем выгрузки из Аналит
* Требуется смотреть по адресу 'h:/СЦ/Исп_показателей_субъекта_Доступность_РЭМД/Сбор данных 2023/Подготовка к ВКС 26.12.2023/И38/для ПРОПП/Аналит доступность'

In [303]:
# Получаем список входных файлов
slide5_input = glob.glob(os.path.join(ANALIT_SLIDE5, "*.xlsx"))

In [304]:
# Объединяем их в один датафрейм
svod_slide5 = []
for my_files in slide5_input:
    next_files = pd.read_excel(my_files, usecols=['Медицинская организация', 'Должность', 'Специальность',  'Врач','Кол.дней ожидания']).dropna()
    svod_slide5.append(next_files)
read_slide5 = pd.concat(svod_slide5, sort=False, axis=0)


In [305]:
read_slide5

,Медицинская организация,Должность,Специальность,Врач,Кол.дней ожидания
6,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-дерматовенеролог,Дерматовенерология,Долотова Татьяна Александровна,0.4
8,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-дерматовенеролог,Дерматовенерология,Долотова Татьяна Александровна,1.2
10,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-дерматовенеролог,Дерматовенерология,Долотова Татьяна Александровна,2
12,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-дерматовенеролог,Дерматовенерология,Долотова Татьяна Александровна,1
14,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-дерматовенеролог,Дерматовенерология,Долотова Татьяна Александровна,0.4
...,...,...,...,...,...
33428,Государственное бюджетное учреждение здравоохр...,врач-офтальмолог,Офтальмология,Азимова Манзура Мукимовна - (врач-офтальмолог),8
33430,Государственное бюджетное учреждение здравоохр...,врач-офтальмолог,Офтальмология,Азимова Манзура Мукимовна - (врач-офтальмолог),нет доступной записи
33432,Государственное бюджетное учреждение здравоохр...,врач-офтальмолог,Офтальмология,Азимова Манзура Мукимовна - (врач-офтальмолог),нет доступной записи
33434,Государственное бюджетное учреждение здравоохр...,врач-офтальмолог,Офтальмология,Азимова Манзура Мукимовна - (врач-офтальмолог),нет доступной записи


###### в системе Аналит для ОБ13 и Перинатального центра есть несколько вариантов наименований. Это может серьезно сказаться на качестве данных. Поэтому я обрабатываю эти наименовния и привожу их к одному варианту.

In [306]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской Области Областная больница №13 (с. Исетское)', 
'Государственное бюджетное учреждение здравоохранения Тюменской области Областная больница № 13 (с.Исетское)',
 read_slide5['Медицинская организация'])

In [307]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница № 13 (с.Исетское)"', 
'Государственное бюджетное учреждение здравоохранения Тюменской области Областная больница № 13 (с.Исетское)',
 read_slide5['Медицинская организация'])

In [308]:
read_slide5['Медицинская организация'] = np.where(read_slide5['Медицинская организация'] == 'Государственное бюджетное учреждение здравоохранения Тюменской области "Перинатальный центр" г. Тюмень', 
'Государственное бюджетное учреждение здравоохранения Тюменской области "Перинатальный центр" (г. Тюмень)',
 read_slide5['Медицинская организация'])

In [309]:
read_slide5['Медицинская организация'].unique()

array(['ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ТЮМЕНСКОЙ ОБЛАСТИ "ОБЛАСТНАЯ БОЛЬНИЦА №4" (Г.ИШИМ)',
       'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница №11" (р.п. Голышманово)',
       'Государственное бюджетное учреждение здравоохранения Тюменской области "Областная больница № 9" (с. Вагай)',
       'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ТЮМЕНСКОЙ ОБЛАСТИ "ОБЛАСТНАЯ БОЛЬНИЦА № 3"',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 1»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 3»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 4»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликлиника № 5»',
       'Государственное автономное учреждение здравоохранения Тюменской области «Городская поликли

In [310]:
read_slide5 = pd.merge(read_slide5, table_analit_doljnost, on=['Должность', 'Специальность'], how='right').dropna()

In [311]:
read_slide5['Кол.дней ожидания']

0                        0.25
1                           1
2                         0.6
3        нет доступной записи
4                           0
                 ...         
48139                       1
48140                     3.2
48141                       3
48142                       2
48143                       1
Name: Кол.дней ожидания, Length: 48143, dtype: object

In [312]:
read_slide5 = read_slide5[['Медицинская организация', 'Должность', 'Врач', 'Кол.дней ожидания']]

In [313]:
# Очистка ячейки, замена на None строковых значений
read_slide5['Кол.дней ожидания'] = np.where(read_slide5['Кол.дней ожидания'] == 'нет доступной записи', None, read_slide5['Кол.дней ожидания'])

In [314]:
# Округляю до наименьшего целого если значение не пусто
read_slide5['Минимум дней'] = read_slide5['Кол.дней ожидания'].map(lambda x: int(math.floor(float(x))) if x is not None else None)

In [315]:
read_slide5

,Медицинская организация,Должность,Врач,Кол.дней ожидания,Минимум дней
0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,0.25,0.0
1,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Голуб Алёна Николаевна,1,1.0
2,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Голуб Владимир Евгеньевич,0.6,0.0
3,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,None,NaN
4,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,0,0.0
...,...,...,...,...,...
48139,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),1,1.0
48140,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),3.2,3.0
48141,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),3,3.0
48142,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),2,2.0


In [316]:
read_slide5['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый',
       'врач-психиатр'], dtype=object)

In [317]:
read_slide5['Минимум дней']

0        0.0
1        1.0
2        0.0
3        NaN
4        0.0
        ... 
48139    1.0
48140    3.0
48141    3.0
48142    2.0
48143    1.0
Name: Минимум дней, Length: 48143, dtype: float64

In [318]:
read_slide5.rename(columns={'Медицинская организация': 'Аналит МО'}, inplace=True)

In [319]:
analit_mo = pd.read_excel(ROOT_PATH + '\\guide\\medicine_organization_3.xlsx')

In [320]:
analit_slide_5 = pd.merge(read_slide5, analit_mo, on='Аналит МО', how='left')

In [321]:
analit_slide_5

,Аналит МО,Должность,Врач,Кол.дней ожидания,Минимум дней,Наименование МО,Организация,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,0.25,0.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
1,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Голуб Алёна Николаевна,1,1.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
2,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Голуб Владимир Евгеньевич,0.6,0.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
3,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,None,NaN,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
4,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,врач-терапевт участковый,Семенова Евгения Викторовна,0,0.0,ГБУЗ ТО «ОБ №4» (г.Ишим),ГБУЗ ТО «ОБ №4»,ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
...,...,...,...,...,...,...,...,...,...,...,...
48138,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),1,1.0,ГБУЗ ТО ОБ №24 (с.Ярково),ГБУЗ ТО «ОБ №24»,ОБ24,"ГБУЗ ТО ""Областная больница №24"" (с.Ярково)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №24"" (С. ЯРКОВО)",ГБУЗ ТО «Областная больница № 24»
48139,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),3.2,3.0,ГБУЗ ТО ОБ №24 (с.Ярково),ГБУЗ ТО «ОБ №24»,ОБ24,"ГБУЗ ТО ""Областная больница №24"" (с.Ярково)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №24"" (С. ЯРКОВО)",ГБУЗ ТО «Областная больница № 24»
48140,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),3,3.0,ГБУЗ ТО ОБ №24 (с.Ярково),ГБУЗ ТО «ОБ №24»,ОБ24,"ГБУЗ ТО ""Областная больница №24"" (с.Ярково)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №24"" (С. ЯРКОВО)",ГБУЗ ТО «Областная больница № 24»
48141,Государственное бюджетное учреждение здравоохр...,врач-психиатр,Макарова Татьяна Викторовна - (Врач-психиатр),2,2.0,ГБУЗ ТО ОБ №24 (с.Ярково),ГБУЗ ТО «ОБ №24»,ОБ24,"ГБУЗ ТО ""Областная больница №24"" (с.Ярково)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №24"" (С. ЯРКОВО)",ГБУЗ ТО «Областная больница № 24»


* Этот код фильтрует DataFrame `analit_slide_5` и оставляет только те строки, где значения столбца 'Должность' находятся в списке `employee_list`.

* Специфически, `analit_slide_5['Должность'].isin(employee_list)` создает булеву серию, которая проверяет, находятся ли значения столбца 'Должность' в списке `employee_list`. Затем, `analit_slide_5[...]` используется для * индексации DataFrame, чтобы оставить только строки, соответствующие `True` значениям в булевой серии.

* Таким образом, после выполнения этого кода, `analit_slide_5` будет содержать только строки, где столбец 'Должность' содержит значения из `employee_list`.


In [322]:
analit_slide_5 = analit_slide_5[analit_slide_5['Должность'].isin(employee_list)]

In [323]:
analit_slide_5['Должность'].unique()

array(['врач-терапевт участковый', 'врач-педиатр участковый',
       'врач общей практики (семейный врач)', 'врач-офтальмолог',
       'врач-оториноларинголог', 'врач-хирург', 'врач-акушер-гинеколог',
       'врач-психиатр-нарколог', 'врач-фтизиатр участковый',
       'врач-фтизиатр', 'врач-стоматолог', 'врач-стоматолог-терапевт',
       'врач-стоматолог детский', 'врач-психиатр детский',
       'врач - детский хирург', 'врач-психиатр детский участковый'],
      dtype=object)

* Этот код определяет пользовательскую функцию `customized_nanmean` для вычисления среднего значения из заданного списка `values`, который может содержать пропущенные значения (`None` или `NaN`).

Сначала код проверяет, являются ли все значения в группе пропущенными (`None`) или NaN (`np.nan`). Для этого используется выражение `all(pd.isna(x) or x is None for x in values)`. Если все значения равны `None` или NaN, функция возвращает `None`, т.е., нет доступных данных для расчета среднего значения и возвращается специальное значение `None`.

В противном случае (если есть хотя бы одно числовое значение), функция использует `np.nanmean(values)` для вычисления среднего значения. Функция `np.nanmean` вычисляет среднее значение, игнорируя пропущенные значения (`NaN`), и возвращает результат с плавающей точкой.

И наконец, результат среднего значения округляется до ближайшего целого числа с помощью `round(..., 0)` и возвращается.

Функция вызывается при построении сводной таблицы, так что в итоге мы получаем массив средних значений за исключением `None`


In [324]:
def customized_nanmean(values):
    # Проверяем, все ли значения в группе равны None или NaN
    if all(pd.isna(x) or x is None for x in values):
        return None
    else:
        return round(np.nanmean(values), 0)

In [325]:
slide_5pivot = pd.pivot_table(analit_slide_5, index='Организация', columns='Должность', values='Минимум дней', aggfunc=customized_nanmean, margins=True, margins_name='Итог').fillna('НДЗ')

In [326]:
slide_5pivot.reset_index(drop=False, inplace=True)

In [327]:
slide_5pivot = slide_5pivot.rename_axis(index=None, columns=None)

In [328]:
slide_5pivot = slide_5pivot[slide_5pivot['Организация'] != 'ГБУЗ ТО «Перинатальный центр»'].reset_index(drop=True)

Вот тут нужно проверить результат на соответсвие с таблицей Ани флягиной чтобы понять должны ли вообще они отправлять данные или нет

In [329]:
uchet_doljnosty = pd.read_excel(ROOT_PATH + '\\guide\\uchet_doljnosty.xlsx').fillna(0)

In [330]:
uchet_doljnosty = uchet_doljnosty.sort_values(by='Организация', ascending=True).reset_index(drop=True)

In [331]:
uchet_doljnosty

,Организация,врач-терапевт участковый,врач общей практики (семейный врач),врач-акушер-гинеколог,врач-педиатр участковый,врач-оториноларинголог,врач-офтальмолог,врач-хирург,врач-психиатр детский,врач-психиатр детский участковый,врач-психиатр-нарколог,врач-стоматолог,врач-стоматолог детский,врач-стоматолог-терапевт,врач-фтизиатр,врач-фтизиатр участковый,врач - детский хирург
0,"ГАУЗ ТО ""ОСП""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,ГАУЗ ТО «ГП №12»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,ГАУЗ ТО «ГП №13»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,ГАУЗ ТО «ГП №17»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,ГАУЗ ТО «ГП №1»,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,ГАУЗ ТО «ГП №3»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,ГАУЗ ТО «ГП №4»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
7,ГАУЗ ТО «ГП №5»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
8,ГАУЗ ТО «ГП №6»,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9,ГАУЗ ТО «ГП №8»,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [332]:
len(slide_5pivot.columns)

18

In [333]:
len(uchet_doljnosty.columns)

17

In [334]:
slide_5pivot

,Организация,врач - детский хирург,врач общей практики (семейный врач),врач-акушер-гинеколог,врач-оториноларинголог,врач-офтальмолог,врач-педиатр участковый,врач-психиатр детский,врач-психиатр детский участковый,врач-психиатр-нарколог,врач-стоматолог,врач-стоматолог детский,врач-стоматолог-терапевт,врач-терапевт участковый,врач-фтизиатр,врач-фтизиатр участковый,врач-хирург,Итог
0,"ГАУЗ ТО ""ОСП""",НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,9.0,НДЗ,10.0,НДЗ,НДЗ,НДЗ,НДЗ,9.0
1,ГАУЗ ТО «ГП №12»,4.0,НДЗ,8.0,6.0,18.0,3.0,НДЗ,НДЗ,НДЗ,13.0,16.0,17.0,3.0,НДЗ,НДЗ,5.0,6.0
2,ГАУЗ ТО «ГП №13»,НДЗ,НДЗ,3.0,6.0,2.0,2.0,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,6.0,3.0,НДЗ,НДЗ,10.0,3.0
3,ГАУЗ ТО «ГП №17»,3.0,НДЗ,8.0,10.0,9.0,2.0,2.0,НДЗ,НДЗ,10.0,НДЗ,НДЗ,2.0,НДЗ,НДЗ,9.0,3.0
4,ГАУЗ ТО «ГП №1»,1.0,1.0,2.0,7.0,5.0,1.0,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,11.0,1.0,НДЗ,НДЗ,10.0,2.0
5,ГАУЗ ТО «ГП №3»,2.0,НДЗ,2.0,5.0,5.0,3.0,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,2.0,НДЗ,НДЗ,5.0,3.0
6,ГАУЗ ТО «ГП №4»,4.0,НДЗ,1.0,3.0,3.0,1.0,2.0,НДЗ,2.0,НДЗ,3.0,4.0,2.0,НДЗ,НДЗ,2.0,2.0
7,ГАУЗ ТО «ГП №5»,2.0,НДЗ,6.0,5.0,15.0,1.0,НДЗ,НДЗ,НДЗ,8.0,10.0,3.0,1.0,НДЗ,НДЗ,7.0,3.0
8,ГАУЗ ТО «ГП №6»,1.0,НДЗ,1.0,1.0,1.0,1.0,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,1.0,НДЗ,НДЗ,1.0,1.0
9,ГАУЗ ТО «ГП №8»,1.0,1.0,3.0,2.0,1.0,1.0,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,НДЗ,1.0,НДЗ,НДЗ,2.0,1.0


In [335]:
slide_5pivot = slide_5pivot[['Организация', 'врач общей практики (семейный врач)', 'врач-акушер-гинеколог', 'врач-оториноларинголог', 
                             'врач-офтальмолог', 'врач-педиатр участковый', 'врач-психиатр-нарколог', 'врач-стоматолог', 
                             'врач-терапевт участковый', 'врач-фтизиатр участковый', 'врач-фтизиатр', 'врач-хирург', 'врач - детский хирург', 
                             'врач-стоматолог-терапевт', 'врач-стоматолог детский', 'врач-психиатр детский', 'врач-психиатр детский участковый']]

In [336]:
# Создание третьего DataFrame, в котором значения из df1 берутся, если значения в duchet_doljnosty != 0
df5 = slide_5pivot.copy()
for column in df5.columns:
    for idx, value in enumerate(df5[column]):
        df5.at[idx, column] = np.where(uchet_doljnosty.at[idx, column] != 0, slide_5pivot.at[idx, column], '-')

In [337]:
df5['Организация'] = df5['Организация'].astype(str)

In [338]:
df5 = pd.merge(df5, sequence_mo, how='right', on='Организация')

In [339]:
df5

,Организация,врач общей практики (семейный врач),врач-акушер-гинеколог,врач-оториноларинголог,врач-офтальмолог,врач-педиатр участковый,врач-психиатр-нарколог,врач-стоматолог,врач-терапевт участковый,врач-фтизиатр участковый,врач-фтизиатр,врач-хирург,врач - детский хирург,врач-стоматолог-терапевт,врач-стоматолог детский,врач-психиатр детский,врач-психиатр детский участковый
0,ГАУЗ ТО «ГП №1»,1.0,2.0,7.0,5.0,1.0,-,-,1.0,-,-,10.0,1.0,11.0,-,-,-
1,ГАУЗ ТО «ГП №12»,-,8.0,6.0,18.0,3.0,-,13.0,3.0,-,-,5.0,4.0,17.0,16.0,НДЗ,-
2,ГАУЗ ТО «ГП №13»,-,3.0,6.0,2.0,2.0,-,НДЗ,3.0,-,-,10.0,-,6.0,-,-,-
3,ГАУЗ ТО «ГП №17»,-,8.0,10.0,9.0,2.0,-,10.0,2.0,-,-,9.0,3.0,-,-,2.0,-
4,ГАУЗ ТО «ГП №3»,-,2.0,5.0,5.0,3.0,-,-,2.0,-,-,5.0,2.0,-,-,-,-
5,ГАУЗ ТО «ГП №4»,-,1.0,3.0,3.0,1.0,-,-,2.0,-,-,2.0,4.0,4.0,3.0,2.0,-
6,ГАУЗ ТО «ГП №5»,-,6.0,5.0,15.0,1.0,-,8.0,1.0,-,-,7.0,2.0,3.0,10.0,-,-
7,ГАУЗ ТО «ГП №6»,-,1.0,1.0,1.0,1.0,-,-,1.0,-,-,1.0,1.0,НДЗ,-,-,-
8,ГАУЗ ТО «ГП №8»,1.0,3.0,2.0,1.0,1.0,-,НДЗ,1.0,-,-,2.0,1.0,НДЗ,-,-,-
9,"ГАУЗ ТО ""ОСП""",-,-,-,-,-,-,9.0,-,-,-,-,-,10.0,-,-,-


##### Расчет 8 столбца первого слайда

###### Вот тут добавляем требование заменять нулевые ставки на 1 

In [340]:
merge_ras_1c_uchet_secret

,Организация,Должность,Проверка,region,mo_oid,mo_short_name,sp_oid,sp_name,mp_id,mp_fio,...,slots_created,slots_booked,slots_free_after_booked,visits_absence,visits_success,Наименование МО,МО,МедОрганизация,Краткое наименование организации,Медицинская организация
10,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
11,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,6,6,0,0,6,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
12,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,1,1,0,0,1,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
13,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,7,7,0,0,7,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
14,ГБУЗ ТО «ОБ №4»,врач-стоматолог детский,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7328,"ГБУЗ ТО ""ОБ №4"" (г.Ишим)",1.2.643.5.1.13.13.12.2.72.7328.0.545743,Детское отделение (стоматологическая поликлиника),6272478571,Коровина Лариса Александровна,...,4,4,0,0,4,ГБУЗ ТО «ОБ №4» (г.Ишим),ОБ4,"ГБУЗ ТО ""Областная больница №4"" (г.Ишим)","ГБУЗ ТО ""ОБ №4"" (Г.ИШИМ)","ГБУЗ ТО ""ОБ №4"" (г.Ишим)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87455,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,12,12,0,0,12,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87456,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87457,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,11,11,0,0,11,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""
87458,ГБУЗ ТО «ОБ №15»,врач-хирург,1.0,Тюменская область,1.2.643.5.1.13.13.12.2.72.7334,ГБУЗ ТО «Областная больница № 15»,1.2.643.5.1.13.13.12.2.72.7334.0.39338,ПОЛИКЛИНИКА,21845278883,Хашимов Абдурахман,...,13,13,0,0,13,ГБУЗ ТО «Областная больница №15» (с.Нижняя Тавда),ОБ15,"ГБУЗ ТО ""Областная больница №15"" (с.Нижняя Тавда)","ГБУЗ ТО ""ОБЛАСТНАЯ БОЛЬНИЦА №15"" (С. НИЖНЯЯ ТА...","ГБУЗ ТО ""ОБ № 15"""


In [341]:
merge_ras_1c_uchet_secret['mp_stavka'] = np.where(merge_ras_1c_uchet_secret['mp_stavka'] == 0, 1, merge_ras_1c_uchet_secret['mp_stavka'])

In [342]:
merge_ras_1c_uchet_data = merge_ras_1c_uchet_secret.copy()

In [343]:
# Расчет 8 столбца первого слайда
slide_1_columns8_data = merge_ras_1c_uchet_data[['Организация', 
                                            'Должность', 
                                            'mp_id', 
                                            'mp_fio', 
                                            'mp_stavka']]
subset_data = slide_1_columns8_data.copy()
subset_data.drop_duplicates(subset=['Организация', 'Должность', 'mp_id'], inplace=True)

# Расчет 8 столбца первого слайда
slide_1_columns8 = pd.pivot_table(subset_data, 
                                  index='Организация', 
                                  values='mp_stavka', 
                                  aggfunc=np.sum, 
                                  margins=True, 
                                  margins_name='Итого').fillna(0).reset_index()
slide_1_columns8.rename(columns={'Организация': 'Наименование МО'}, inplace=True)

##### Итоговый расчет первого слайда

In [344]:
# Итоговый расчет первого слайда
# Объединяем 
slide_1_all = [slide_1_column1, slide_1_column2, slide_1_column3, slide1_column4, slide_1_column6, slide_1_column7, slide_1_columns8]
df = reduce(lambda left, right: pd.merge(left,right,on=['Наименование МО'], how='outer'), slide_1_all)

df.rename(columns={'Всего': 'Кол-во физ-лиц в ФРМР', 'Итого': 'Кол-во физ-лиц в ФРМР за минусом ДО'}, inplace=True)
df = df.fillna(0)

# ##### Вот тут я пересчитываю количество физических лиц за минусом ДО еще раз
df['Кол-во физ-лиц в ФРМР за минусом ДО'] = df['Кол-во физ-лиц в ФРМР'] - df['Кол-во физ-лиц в ФРМР в ДО']
df['Доля физ. лиц с расписанием от общего количества физ лиц'] = round(df['Кол-во физ-лиц на которые заведено расписание'] / df['Кол-во физ-лиц в ФРМР за минусом ДО'] * 100, 1)
df.replace([np.inf , -np.inf ], 0 , inplace= True)
df = df[df['Наименование МО'] != 'Итого']
df = df.sort_values(by='Доля физ. лиц с расписанием от общего количества физ лиц', ascending=False)
df.reset_index(drop=True)
df = df.set_index('Наименование МО')
df.loc['Итого']=df.sum()
df = df.reset_index()
# Значение для поиска
index_values = 'Итого'
# Находим индекс строки с заданным значением в 'Столбец1'
index_values_value = df.loc[df['Наименование МО'] == index_values].index[0]
# Вычисляем среднее значение по столбцу 'Столбец2'
mean_procent = df['Доля физ. лиц с расписанием от общего количества физ лиц'].median()
# Изменяем значение ячейки на среднее значение
df.loc[index_values_value , 'Доля физ. лиц с расписанием от общего количества физ лиц'] = mean_procent
df['Доля физ. лиц с расписанием от общего количества физ лиц'] = round(df['Доля физ. лиц с расписанием от общего количества физ лиц'], 1)
df.rename(columns={'Кол-во должностей из 14': 'Кол-во введенных должностей в ШР МО из учитываемых 14',
                   'Кол-во физ-лиц в ФРМР за минусом ДО': 'Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
                   'Доля физ. лиц с расписанием от общего количества физ лиц': 'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
                   'Кол-во физ-лиц в ФРМР в ДО': 'Кол-во физ-лиц в ФРМР находящихся в отпуске по беременности и родам и по уходу за ребенком', 
                   'mp_stavka': 'Кол-во ставок на которое сформировано расписание'}, inplace=True)
df = df[['Наименование МО',
       'Кол-во введенных должностей в ШР МО из учитываемых 14',
       'Кол-во физ-лиц в ФРМР',
       'Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком',
       'Кол-во физ-лиц на которые заведено расписание',
       'Кол-во физ-лиц в ФРМР находящихся в отпуске по беременности и родам и по уходу за ребенком',
       'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком', 
       'Кол-во ставок в ШР МО', 
       'Кол-во ставок на которое сформировано расписание']]
df['Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком'] = round(df['Кол-во физ-лиц на которые заведено расписание'] / df['Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком'] * 100, 1)

In [345]:
df

,Наименование МО,Кол-во введенных должностей в ШР МО из учитываемых 14,Кол-во физ-лиц в ФРМР,Кол-во физ-лиц в ФРМР за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком,Кол-во физ-лиц на которые заведено расписание,Кол-во физ-лиц в ФРМР находящихся в отпуске по беременности и родам и по уходу за ребенком,Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком,Кол-во ставок в ШР МО,Кол-во ставок на которое сформировано расписание
0,ГБУЗ ТО «ОКПБ»,1.0,7.0,6.0,7.0,1.0,116.7,6.50,5.50
1,ГБУЗ ТО «Родильный дом №3»,1.0,11.0,10.0,11.0,1.0,110.0,9.25,8.25
2,ГБУЗ ТО «ОКБ №2»,8.0,62.0,57.0,60.0,5.0,105.3,57.50,59.00
3,ГБУЗ ТО «ОБ №3»,11.0,102.0,90.0,94.0,12.0,104.4,85.50,71.00
4,"ГАУЗ ТО ""ОСП""",2.0,60.0,54.0,55.0,6.0,101.9,54.25,49.75
5,ГБУЗ ТО «ОКФЦ»,1.0,27.0,25.0,25.0,2.0,100.0,25.00,25.50
6,ГАУЗ ТО «ГП №8»,10.0,103.0,86.0,86.0,17.0,100.0,84.25,82.75
7,ГБУЗ ТО «Роддом № 2»,1.0,11.0,10.0,10.0,1.0,100.0,10.00,8.25
8,ГБУЗ ТО «ОБ №11»,11.0,72.0,70.0,69.0,2.0,98.6,70.75,62.25
9,ГАУЗ ТО «ГП №12»,11.0,173.0,151.0,146.0,22.0,96.7,147.00,141.50


In [346]:
df['Кол-во ставок в ШР МО'] = df['Кол-во ставок в ШР МО'].round(2)

###### Создаем презентацию

In [347]:
# Создаем презентацию
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(9)
slide_width = prs.slide_width
slide_height = prs.slide_height

##### Создаю первый слайд

In [348]:

# Создаю первый слайд
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(blank_slide_layout)
# Создаем заголовок
title_slide = slide.shapes.title
# Установка цвета фона заголовка
title_slide.fill.solid()
title_slide.fill.fore_color.rgb = color_title
title_slide.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period_last}) «ОХВАТ РАСПИСАНИЕМ СОТРУДНИКОВ»'
title_slide.height = Inches(0.8)
title_slide.width = Inches(16)
title_slide.top = Inches(0.1)
title_slide.left = Inches(0)
# Устанавливаем размер шрифта
title_slide.text_frame.paragraphs[0].runs[0].font.size = Pt(10)
# Устанавливаем цвет шрифта
title_slide.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
#Создаем макет табличной формы
left = Inches(0)
width = int(slide_width * 1)
height = int (slide_height * 0.7)
top = Inches(1)
shape = slide.shapes.add_table(30, 9, left, top, width, height)
table_placeholder = slide.shapes[0]
# Создаем таблицу
table = shape.table
# Уменьшаем отступы в ячейках таблицы
for row in table.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df.columns):
    table.cell(0, i).text = column
    cell = table.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table.cell(i, j).text = str(value)

for i, row in enumerate(df.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'

# Слайд 1 Столбец 7
column_index = 6
column_name = df.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 80 <= value < 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

In [349]:
df2

,Наименование МО,"Доля фактического количества слотов от расчетных, не менее 80%","Доля конкурентных слотов от общего количества слотов, не менее 65%","Доля фактического количества слотов от расчетных у врачей участковой службы, не менее 80%","Доля конкурентных слотов ПЛАН составленного расписания, не менее 65%","Доля конкурентных слотов Фактического расписания участковой службы от общего количества слотов, не менее 65%"
0,ГАУЗ ТО «ГП №1»,67.47,58.16,70.59,65.54,58.35
1,ГАУЗ ТО «ГП №12»,61.63,48.91,61.28,72.11,53.68
2,ГАУЗ ТО «ГП №13»,81.46,57.84,80.96,66.04,59.97
3,ГАУЗ ТО «ГП №17»,68.05,61.13,67.25,75.73,64.37
4,ГАУЗ ТО «ГП №3»,71.46,53.06,68.35,67.88,53.08
5,ГАУЗ ТО «ГП №4»,58.96,55.43,61.02,65.37,52.33
6,ГАУЗ ТО «ГП №5»,72.65,62.14,76.29,67.88,64.38
7,ГАУЗ ТО «ГП №6»,75.61,64.99,76.87,70.74,63.60
8,ГАУЗ ТО «ГП №8»,77.04,58.38,72.61,75.60,60.74
9,"ГАУЗ ТО ""ОСП""",87.23,51.85,0.00,64.30,0.00


##### Создание слайда 2

In [350]:
# Создание слайда 2
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide2 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height

slide2.slide_width = slide_width
slide2.slide_height = slide_height
# Создаем заголовок
title_slide2 = slide2.shapes.title
title_slide2.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period}) «ПОЛНОТА ВЕДЕНИЯ РАСПИСАНИЯ И КОНКУРЕНТНОСТЬ»'
title_slide2.height = Inches(0.9)
title_slide2.width = Inches(16)
title_slide2.top = Inches(0.1)
title_slide2.left = Inches(0)
# Установка цвета фона заголовка
title_slide2.fill.solid()
title_slide2.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide2.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide2.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0.1)
width = int(slide_width * 0.98)
height = int (slide_height * 0.8)
top = Inches(1.1)
shape2 = slide2.shapes.add_table(30, 6, left, top, width, height)
table_placeholder2 = slide2.shapes[0]
# Создаем таблицу
table2 = shape2.table
# Уменьшаем отступы в ячейках таблицы
for row in table2.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df2.columns):
    table2.cell(0, i).text = column
    cell = table2.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(12)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df2.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table2.cell(i, j).text = str(value)

for i, row in enumerate(df2.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table2.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'
# Столбец 6
column_index = 5
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow
    elif value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  


# Столбец 5
column_index = 4
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  

# Столбец 4
column_index = 3
column_name = df2.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df2[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Желтый цвет
    elif value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

# Столбец 3
column_index = 2
column_name = df2.columns[column_index]

# Получение соответствующего столбца из DataFrame
column = df2[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет
# Столбец 2
column_index = 1
column_name = df2.columns[column_index]

# Получение соответствующего столбца из DataFrame
column = df2[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table2.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow 
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [351]:
df3

,Наименование МО,Занятость слотов всего (доля от общего числа созданных и количества занятых),Доля неявок от общего числа записей,"Доля пациентов, принятых без предварительной записи",Доля посещений на дому
0,ГАУЗ ТО «ГП №1»,81.14,3.10,3.14,66.7
1,ГАУЗ ТО «ГП №12»,89.43,2.99,9.37,2.8
2,ГАУЗ ТО «ГП №13»,81.73,4.86,11.24,0.0
3,ГАУЗ ТО «ГП №17»,91.43,10.82,13.32,0.4
4,ГАУЗ ТО «ГП №3»,86.24,4.34,13.22,0.0
5,ГАУЗ ТО «ГП №4»,85.59,6.46,2.35,68.8
6,ГАУЗ ТО «ГП №5»,85.64,9.23,3.06,0.0
7,ГАУЗ ТО «ГП №6»,76.43,6.70,2.41,8.6
8,ГАУЗ ТО «ГП №8»,74.11,8.99,18.68,10.5
9,ГБУЗ ТО «ОБ №11»,73.25,2.72,10.54,0.9


##### Создаем слайд 3

In [352]:
# Создаем слайд 3
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide3 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide3.slide_width = slide_width
slide3.slide_height = slide_height

# Создаем заголовок
title_slide3 = slide3.shapes.title
title_slide3.text = f'ПОКАЗАТЕЛИ ОРГАНИЗАЦИИ ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period}) «ЭФФЕКТИВНОСТЬ ФОРМИРОВАНИЯ РАСПИСАНИЯ»'
title_slide3.height = Inches(0.9)
title_slide3.width = Inches(16)
title_slide3.top = Inches(0.1)
title_slide3.left = Inches(0)

# Установка цвета фона заголовка
title_slide3.fill.solid()
title_slide3.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide3.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide3.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0.1)
width = int(slide_width * 0.98)
height = int (slide_height * 0.85)
top = Inches(1.1)
shape3 = slide3.shapes.add_table(30, 5, left, top, width, height)
table_placeholder3 = slide3.shapes[0]
# Создаем таблицу
table3 = shape3.table
# Уменьшаем отступы в ячейках таблицы
for row in table3.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df3.columns):
    table3.cell(0, i).text = column
    cell = table3.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона заголовка

for i, row in enumerate(df3.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table3.cell(i, j).text = str(value)

for i, row in enumerate(df3.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table3.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'


# Столбец 2
column_name = 'Занятость слотов всего (доля от общего числа созданных и количества занятых)'
column_index = 1

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 50 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет
# Столбец 3
column_name = 'Доля неявок от общего числа записей'
column_index = 2

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 20:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 20 <= value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет


# Столбец 4
column_name = 'Доля пациентов, принятых без предварительной записи'
column_index = 3

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 20:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif 20 <= value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет


# Столбец 5
column_name = 'Доля посещений на дому'
column_index = 4

# Получение соответствующего столбца из DataFrame
column = df3[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table3.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value == 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  # Нейтральный цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [353]:
df4

,Наименование МО,Число лиц включенных в лист ожидания,Число лиц исключенных из листа ожидания по причине записи,Количество конкурентной записи,"Доля лиц, включенных в лист ожидания от общего количества конкурентной записи",Движение лиц в листе ожидания
0,ГБУЗ ТО «ОБ №15»,11.0,0.0,1450.0,0.76,100.00
1,0,5.0,0.0,0.0,inf,100.00
2,ГБУЗ ТО «ОБ №9»,2.0,0.0,1198.0,0.17,100.00
3,ГБУЗ ТО «ОБ №19»,65.0,3.0,6502.0,1.00,95.38
4,ГАУЗ ТО «ГП №8»,383.0,24.0,6784.0,5.65,93.73
5,ГБУЗ ТО «ОБ №4»,91.0,7.0,7539.0,1.21,92.31
6,ГБУЗ ТО «ОБ №12»,9.0,1.0,4172.0,0.22,88.89
7,ГБУЗ ТО «ОБ №24»,25.0,3.0,1193.0,2.10,88.00
8,ГБУЗ ТО «ОБ №23»,96.0,12.0,3030.0,3.17,87.50
9,ГБУЗ ТО «ОКБ №2»,63.0,8.0,2628.0,2.40,87.30


In [354]:
df4 = df4[df4['Наименование МО'] != 0]

##### Создаем 4 слайд

In [355]:
# Создаем 4 слайд
# Указываем макет создаем слайд настраиваем параменты слайда
blank_slide_layout = prs.slide_layouts[5]
slide4 = prs.slides.add_slide(blank_slide_layout)

slide_width = prs.slide_width
slide_height = prs.slide_height

slide4.slide_width = slide_width
slide4.slide_height = slide_height

# Создаем заголовок
title_slide4 = slide4.shapes.title
title_slide4.text = f'ЭФФЕКТИВНОСТЬ ОРГАНИЗАЦИИ ОБРАБОТКИ И РЕГИСТРАЦИИ ИНФОРМАЦИИ ОБ ОБРАЩЕНИЯХ ЗАПИСИ ОБРАТНАЯ СВЯЗЬ ПО ЗАПИСИ В ОТЧЕТНОМ ПЕРИОДЕ ({period})'
title_slide4.height = Inches(0.9)
title_slide4.width = Inches(16)
title_slide4.top = Inches(0.1)
title_slide4.left = Inches(0)

# Установка цвета фона заголовка
title_slide4.fill.solid()
title_slide4.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide4.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide4.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral
# Создаем макет табличной формы
left = Inches(0)
width = int(slide_width)
height = int (slide_height * 0.8)
top = Inches(1.1)
shape4 = slide4.shapes.add_table(30, 6, left, top, width, height)
table_placeholder4 = slide4.shapes[0]
# Создаем таблицу
table4 = shape4.table
# Уменьшаем отступы в ячейках таблицы
for row in table4.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df4.columns):
    table4.cell(0, i).text = column
    cell = table4.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(12)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки заголовка

for i, row in enumerate(df4.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table4.cell(i, j).text = str(value)

for i, row in enumerate(df4.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table4.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = Inches(0.3)  # Ширина ячейки
        cell.height = Inches(0.6)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(10)
        p.font.name = 'Calibri'




In [356]:
# Имя столбца для итерации
column_name = 'Движение лиц в листе ожидания'

column_index = 5

# Получение соответствующего столбца из DataFrame
column = df4[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table4.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value <= 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [357]:
# Имя столбца для итерации
column_name = 'Доля лиц, включенных в лист ожидания от общего количества конкурентной записи'

column_index = 4

# Получение соответствующего столбца из DataFrame
column = df4[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table4.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value < 1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  # Зеленый цвет
    elif value >= 1 <= 5:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow # Зеленый цвет
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red  # Красный цвет

In [358]:
df5

,Организация,врач общей практики (семейный врач),врач-акушер-гинеколог,врач-оториноларинголог,врач-офтальмолог,врач-педиатр участковый,врач-психиатр-нарколог,врач-стоматолог,врач-терапевт участковый,врач-фтизиатр участковый,врач-фтизиатр,врач-хирург,врач - детский хирург,врач-стоматолог-терапевт,врач-стоматолог детский,врач-психиатр детский,врач-психиатр детский участковый
0,ГАУЗ ТО «ГП №1»,1.0,2.0,7.0,5.0,1.0,-,-,1.0,-,-,10.0,1.0,11.0,-,-,-
1,ГАУЗ ТО «ГП №12»,-,8.0,6.0,18.0,3.0,-,13.0,3.0,-,-,5.0,4.0,17.0,16.0,НДЗ,-
2,ГАУЗ ТО «ГП №13»,-,3.0,6.0,2.0,2.0,-,НДЗ,3.0,-,-,10.0,-,6.0,-,-,-
3,ГАУЗ ТО «ГП №17»,-,8.0,10.0,9.0,2.0,-,10.0,2.0,-,-,9.0,3.0,-,-,2.0,-
4,ГАУЗ ТО «ГП №3»,-,2.0,5.0,5.0,3.0,-,-,2.0,-,-,5.0,2.0,-,-,-,-
5,ГАУЗ ТО «ГП №4»,-,1.0,3.0,3.0,1.0,-,-,2.0,-,-,2.0,4.0,4.0,3.0,2.0,-
6,ГАУЗ ТО «ГП №5»,-,6.0,5.0,15.0,1.0,-,8.0,1.0,-,-,7.0,2.0,3.0,10.0,-,-
7,ГАУЗ ТО «ГП №6»,-,1.0,1.0,1.0,1.0,-,-,1.0,-,-,1.0,1.0,НДЗ,-,-,-
8,ГАУЗ ТО «ГП №8»,1.0,3.0,2.0,1.0,1.0,-,НДЗ,1.0,-,-,2.0,1.0,НДЗ,-,-,-
9,"ГАУЗ ТО ""ОСП""",-,-,-,-,-,-,9.0,-,-,-,-,-,10.0,-,-,-


##### Создаем пятый слайд

In [359]:
# Создаем пятый слайд
blank_slide_layout = prs.slide_layouts[5]
slide5 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide5.slide_width = slide_width
slide5.slide_height = slide_height
# Создаем заголовок
title_slide5 = slide5.shapes.title
title_slide5.text = f'СРЕДНЯЯ ДЛИТЕЛЬНОСТЬ ОЖИДАНИЯ (кол-во дней) В ОТЧЕТНОМ ПЕРИОДЕ ({period})'
title_slide5.height = Inches(0.6)
title_slide5.width = Inches(16)
title_slide5.top = Inches(0.1)
title_slide5.left = Inches(0)
# Установка цвета фона заголовка
title_slide5.fill.solid()
title_slide5.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide5.text_frame.paragraphs[0].runs[0].font.size = Pt(14)
# Устанавливаем цвет шрифта
title_slide5.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
# Создаем макет табличной формы
left = Inches(0.5)
width = int(slide_width * 0.8)
height = int(slide_height * 0.8)
top = Inches(0.7)
shape5 = slide5.shapes.add_table(30, 17, left, top, width, height)
table_placeholder5 = slide5.shapes[0]
# Создаем таблицу
table5 = shape5.table
# Уменьшаем отступы в ячейках таблицы
for row in table5.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df5.columns):
    table5.cell(0, i).text = column
    cell = table5.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df5.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table5.cell(i, j).text = str(value)

for i, row in enumerate(df5.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table5.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = int(slide_width * 0.05)  # Ширина ячейки
        cell.height = int(slide_height * 0.02)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(9)
        p.font.name = 'Calibri'

# Итерация по ячейкам таблицы и раскраска по условию
for row in table5.rows:
    for cell in row.cells:
        # Условие для раскраски ячеек в красный цвет, если значение ячейки равно "НДЗ"
        if cell.text == "НДЗ":
            fill = cell.fill
            fill.solid()
            fill.fore_color.rgb = color_red  # Красный цвет

# Получаем объект столбца по его номеру
table5.columns[0].width = Inches(2)

exception_list_1 = ['0.0', '1.0', '-']
exception_list_2 = ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0','-']
# ###### Раскраска по столбцам
# Столбец 1
column_name = 'врач общей практики (семейный врач)'
column_index = 1

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral


In [360]:

# Столбец 2
column_name = 'врач-акушер-гинеколог'

column_index = 2

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 3
column_name = 'врач-оториноларинголог'

column_index = 3

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 4
column_name = 'врач-офтальмолог'
column_index = 4

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 5
column_name = 'врач-педиатр участковый'
column_index = 5

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 6
column_name = 'врач-психиатр-нарколог'
column_index = 6

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 7
column_name = 'врач-стоматолог'
column_index = 7

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 8
column_name = 'врач-терапевт участковый'
column_index = 8

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_1:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 9
column_name = 'врач-фтизиатр участковый'
column_index = 9

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 10
column_name = 'врач-фтизиатр'
column_index = 10

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 11
column_name = 'врач-хирург'
column_index = 11

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 12
column_name = 'врач - детский хирург'
column_index = 12

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 13
column_name = 'врач-стоматолог-терапевт'
column_index = 13

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 14
column_name = 'врач-стоматолог детский'
column_index = 14

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral


# Столбец 15
column_name = 'врач-психиатр детский'
column_index = 15

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

# Столбец 16
column_name = 'врач-психиатр детский участковый'
column_index = 16

# Получение соответствующего столбца из DataFrame
column = df5[column_name]

# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table5.cell(i, column_index)
    
    # Раскраска ячейки на основе значения из DataFrame
    if value not in exception_list_2:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral

##### Формируем колонки 6 слайда

In [361]:
# Формируем колонки 6 слайда
sequence_mo3 = sequence_mo2.copy()
sequence_mo3.loc[len(sequence_mo3)] = 'Итого'
slide_6_column1 = df[['Наименование МО', 'Доля физ. лиц с расписанием от общего количества физ лиц за минусом находящихся в отпуске по беременности и родам и по уходу за ребенком']]
slide_6_column1 = pd.merge(sequence_mo3, slide_6_column1, how='left', on='Наименование МО')
slide_6_column2 = itog_slots
slide_6_column2.rename(columns={'Организация': 'Наименование МО'}, inplace=True)
slide_6_column2 = pd.merge(sequence_mo3, slide_6_column2, how='left', on='Наименование МО')
slide_6_column3 = df2[['Наименование МО', 'Доля конкурентных слотов от общего количества слотов, не менее 65%']]
slide_6_column4 = result_slide3[['Наименование МО', 'Занятость слотов всего (доля от общего числа созданных и количества занятых)']]
slide_6_column4 = pd.merge(sequence_mo3, slide_6_column4, how='left', on='Наименование МО')
slide_6_column5 = result_slide3[['Наименование МО', 
                                 'Доля пациентов, принятых без предварительной записи']]
slide_6_column5 = pd.merge(sequence_mo3, slide_6_column5, how='left', on='Наименование МО')
slide_6_column6 = result_slide4[['Наименование МО', 'Движение лиц в листе ожидания']]
slide_6_column6 = pd.merge(sequence_mo3, slide_6_column6, how='left', on='Наименование МО')
# Агрегируем данные по колонкам
slide_6_all = [slide_6_column1, 
               slide_6_column2, 
               slide_6_column3, 
               slide_6_column4, 
               slide_6_column5, 
               slide_6_column6]
df6 = reduce(lambda left, right: pd.merge(left,right,on=['Наименование МО'], how='outer'), slide_6_all)

##### Создаем шестой слайд

In [362]:
#Создаем шестой слайд

# Указываем макет создаем слайд настраиваем параменты слайда

blank_slide_layout = prs.slide_layouts[5]
slide6 = prs.slides.add_slide(blank_slide_layout)
slide_width = prs.slide_width
slide_height = prs.slide_height
slide6.slide_width = slide_width
slide6.slide_height = slide_height
# Создаем заголовок

title_slide6 = slide6.shapes.title

title_slide6.text = 'СИСТЕМА КОНТРОЛЯ - РАБОТА С ВЫРАВНИВАНИЕМ ПОКАЗАТЕЛЕЙ'
title_slide6.height = Inches(0.6)
title_slide6.width = Inches(16)
title_slide6.top = Inches(0.1)
title_slide6.left = Inches(0)
# Установка цвета фона заголовка
title_slide6.fill.solid()
title_slide6.fill.fore_color.rgb = color_title
# Устанавливаем размер шрифта
title_slide6.text_frame.paragraphs[0].runs[0].font.size = Pt(12)
# Устанавливаем цвет шрифта
title_slide6.text_frame.paragraphs[0].runs[0].font.color.rgb = color_neutral 
# Создаем макет табличной формы
left = Inches(0.5)
width = int(slide_width * 0.8)
height = int(slide_height * 0.8)
top = Inches(0.7)
shape6 = slide6.shapes.add_table(30, 7, left, top, width, height)
table_placeholder6 = slide6.shapes[0]
# Создаем таблицу
table6 = shape6.table
# Уменьшаем отступы в ячейках таблицы
for row in table6.rows:
    for cell in row.cells:
        cell.margin_top = 0
        cell.margin_bottom = 0
# Заполняем заголовки столбцов
for i, column in enumerate(df6.columns):
    table6.cell(0, i).text = column
    cell = table6.cell(0, i)
    cell.text_frame.clear()  # Очистка содержимого ячейки
    cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
    p = cell.text_frame.paragraphs[0]
    p.text = column
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(10)
    p.font.name = 'Calibri'
    cell.fill.solid()
    cell.fill.fore_color.rgb = color_header  # Задание цвета фона ячейки

for i, row in enumerate(df6.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        table6.cell(i, j).text = str(value)

for i, row in enumerate(df6.itertuples(), start=1):
    for j, value in enumerate(row[1:], start=0):
        cell = table6.cell(i, j)
        cell.text_frame.clear()  # Очистка содержимого ячейки
        cell.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE  # Масштабирование текста
        cell.width = int(slide_width * 0.05)  # Ширина ячейки
        cell.height = int(slide_height * 0.02)  # Высота ячейки
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_neutral  # Задание цвета фона ячейки
        p = cell.text_frame.paragraphs[0]
        p.text = str(value)
        p.alignment = PP_ALIGN.CENTER
        p.font.size = Pt(9)
        p.font.name = 'Calibri'

# Слайд 1 Столбец 1
column_index = 1
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 80 <= value < 100:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 2
column_index = 2
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 3
column_index = 3
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 60 <= value < 65:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 4
column_index = 4
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value >= 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green  
    elif 50 <= value < 80:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red

# Слайд 1 Столбец 5
column_index = 5
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value > 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red
    elif 20 < value <= 30:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_yallow  
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green

# Слайд 1 Столбец 6
column_index = 6
column_name = df6.columns[column_index]
# Получение соответствующего столбца из DataFrame
column = df6[column_name]
# Итерирование по значениям столбца
for i, value in enumerate(column, start=1):
    # Получение ячейки таблицы на текущей позиции столбца
    cell = table6.cell(i, column_index)
    # Раскраска ячейки на основе значения из DataFrame
    if value <= 0:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_green    
    else:
        cell.fill.solid()
        cell.fill.fore_color.rgb = color_red



In [363]:
prs.save('C:\\Users\\propp_sa\\Desktop\\VScode\\prezentation\\output\\ПрезентацияИ38.pptx')

##### Таблица по штатным единицам

* Создаем объединенный файл на основе выгрузки ФРМР

In [471]:
merge_frmr_to_month = frmr_all_uchet.copy()
merge_frmr_to_month = merge_frmr_to_month[['Организация', 'Структурное подразделение', 'Должность', 'Штатных единиц', 'Занятых штатных единиц', 'В т.ч. внешнее совместительство', 'Вакансий']]
merge_frmr_to_month.to_excel('C:\\Users\\propp_sa\\Desktop\\VScode\\prezentation\\output\\Объединенный.xlsx', index=False)

* Считаем штатные единицы и вакансии

In [473]:
slide7_input = frmr_all_uchet.copy()
slide7_input = slide7_input[['Организация', 'Должность', 'Штатных единиц', 'Вакансий']]
# Группируем данные по столбцам 'Организация' и 'Должность' и суммируем значения в столбце 'Занятых штатных единиц'
slide7_post = slide7_input.groupby(['Организация', 'Должность'])['Штатных единиц'].sum().reset_index()
# Группируем данные по столбцам 'Организация' и 'Должность' и суммируем значения в столбце 'Вакансий'
slide7_vacancy = slide7_input.groupby(['Организация', 'Должность'])['Вакансий'].sum().reset_index()
slide7_vp = slide7_input.groupby(['Организация', 'Должность'])['Штатных единиц', 'Вакансий'].sum().reset_index()

C:\Users\propp_sa\AppData\Local\Temp\ipykernel_3220\2551933847.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  slide7_vp = slide7_input.groupby(['Организация', 'Должность'])['Штатных единиц', 'Вакансий'].sum().reset_index()


* Объединяем это с данными расписания

In [477]:
slide7_tree = pd.merge(slide7_vp, employee_with_schedule, how='left', on=['Организация', 'Должность']).fillna(0)
slide7_tree['Дата'] = end_date
slide7_tree['Дата'] = pd.to_datetime(slide7_tree['Дата'], format='%Y-%m-%d')
slide7_tree['Дата'] = slide7_tree['Дата'] + pd.Timedelta(days=1)
slide7_tree.rename(columns={'кол-во сотрудников с сформированным расписанием на 7 дней': 'Сотрудников с расписанием на 7 дней'}, inplace=True)

In [482]:
slide7_tree['День'] = slide7_tree['Дата'].dt.day

In [485]:
slide7_tree['Месяц'] = slide7_tree['Дата'].dt.month

In [486]:
slide7_tree['Год'] = slide7_tree['Дата'].dt.year

In [487]:
slide7_tree

,Организация,Должность,Штатных единиц,Вакансий,Сотрудников с расписанием на 7 дней,Слотов на 7 дней,Слотов первичного приёма на 7 дней,Дата,День,Месяц,Год
0,"ГАУЗ ТО ""ОСП""",врач-стоматолог,30.50,1.00,29.0,1398.0,919.0,2024-02-26,26,2,2024
1,"ГАУЗ ТО ""ОСП""",врач-стоматолог-терапевт,29.75,5.00,26.0,1000.0,623.0,2024-02-26,26,2,2024
2,ГАУЗ ТО «ГП №12»,врач - детский хирург,3.00,0.00,3.0,157.0,103.0,2024-02-26,26,2,2024
3,ГАУЗ ТО «ГП №12»,врач-акушер-гинеколог,17.25,3.50,15.0,757.0,530.0,2024-02-26,26,2,2024
4,ГАУЗ ТО «ГП №12»,врач-оториноларинголог,9.00,0.50,8.0,639.0,452.0,2024-02-26,26,2,2024
...,...,...,...,...,...,...,...,...,...,...,...
221,ГБУЗ ТО «ОКПБ»,врач-психиатр детский участковый,8.00,1.50,7.0,330.0,225.0,2024-02-26,26,2,2024
222,ГБУЗ ТО «ОКФЦ»,врач-фтизиатр участковый,25.00,0.00,25.0,1356.0,915.0,2024-02-26,26,2,2024
223,ГБУЗ ТО «ОНД»,врач-психиатр-нарколог,12.00,0.00,12.0,633.0,424.0,2024-02-26,26,2,2024
224,ГБУЗ ТО «Роддом № 2»,врач-акушер-гинеколог,10.00,0.00,10.0,1014.0,743.0,2024-02-26,26,2,2024


In [488]:
slide7_tree.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Организация                          226 non-null    object        
 1   Должность                            226 non-null    object        
 2   Штатных единиц                       226 non-null    float64       
 3   Вакансий                             226 non-null    float64       
 4   Сотрудников с расписанием на 7 дней  226 non-null    float64       
 5   Слотов на 7 дней                     226 non-null    float64       
 6   Слотов первичного приёма на 7 дней   226 non-null    float64       
 7   Дата                                 226 non-null    datetime64[ns]
 8   День                                 226 non-null    int64         
 9   Месяц                                226 non-null    int64         
 10  Год           

In [489]:
slide7_tree.to_excel('C:\\Users\\propp_sa\\Desktop\\VScode\\prezentation\\output\\resultI38.xlsx', index=False)